In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 10000)

import pandas_profiling
from wordcloud import WordCloud

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from matplotlib.pyplot import figure

import requests
import json

import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

%matplotlib inline
import missingno as msno

rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

#### merge 전략
* df_추정매출 에 df_매출18 중 서비스업종코드 추가분 만 merge
* (1) 매출18 merge (상권매출에서 골목상권 drop 후 merge => 중복 상권 제거)
* (2) 추정매출 업종에 없는 업종 18매출에서 찾기
* (3) 업종 총 수 81개 목표 (현재 63개 확보)

In [19]:
df_점포이력최종 = pd.read_pickle('df_점포이력최종.pki')
df_점포이력최종.columns

Index(['adres', '도로명', '상권_코드_명', 'svcIndutyCdNm', '소분류업종', 'storNm',
       'totalCount', 'coordx', 'coordy', 'x', 'y', 'opbizYm', 'clsbizYm',
       '영업일계산기준일', '영업일수'],
      dtype='object')

In [20]:
df_점포이력최종 = df_점포이력최종.rename(columns={'svcIndutyCdNm':'서비스_업종_코드_명'})
df_점포이력최종

,adres,도로명,상권_코드_명,서비스_업종_코드_명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,영업일수
0,서울특별시 강서구 초원로6길 10 (방화동),초원로6길,개화동로23길,자동차부품,자동차부품,두영시트,2,183007,452002,126.807594,37.567293,2009-05-01,,2020-12-01,4232 days
1,"서울특별시 강서구 초원로6길 10, 지하101호,1층전체 (방화2동)",초원로6길,개화동로23길,자동차수리,자동차수리,휠스타,2,183007,452002,126.807594,37.567293,2011-01-01,201604,2016-04-01,1917 days
2,서울특별시 강서구 개화동로27길 23 (방화2동),개화동로27길,개화동로25길,조명용품,조명용품,코스코,3,183029,451680,126.807851,37.564392,2002-03-01,201912,2019-12-01,6484 days
3,서울특별시 강서구 개화동로27길 23 (방화동),개화동로27길,개화동로25길,컴퓨터및주변장치판매,컴퓨터및주변장치판매,디엔에스,3,183029,451680,126.807851,37.564392,2002-11-01,201704,2017-04-01,5265 days
4,서울특별시 강서구 개화동로27길 23 (방화동),개화동로27길,개화동로25길,미곡판매,미곡판매,대원농원,3,183029,451680,126.807851,37.564392,2003-08-01,200402,2004-02-01,184 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216715,서울특별시 강동구 상일로5길 3 (상일동),상일로5길,천호대로219길,자동차미용,자동차미용,KP,8,215275,449959,127.172809,37.548918,2008-07-01,200901,2009-01-01,184 days
1216716,서울특별시 강동구 상일로5길 3 (상일동),상일로5길,천호대로219길,전자상거래업,전자상거래업,한솔소방ENG,8,215275,449959,127.172809,37.548918,2003-12-01,200501,2005-01-01,397 days
1216717,"서울특별시 강동구 천호대로221길 6, 지층 (상일동)",천호대로221길,천호대로219길,철물점,철물점,동산종합테크,3,215286,449781,127.172929,37.547314,2017-01-01,,2020-12-01,1430 days
1216718,서울특별시 강동구 천호대로221길 6 (상일동),천호대로221길,천호대로219길,제과점,제과점,두리앙제과점,3,215286,449781,127.172929,37.547314,1998-01-01,,2020-12-01,8370 days


In [21]:
print(df_점포이력최종.columns)
print(df_추정매출.columns)

Index(['adres', '도로명', '상권_코드_명', '서비스_업종_코드_명', '소분류업종', 'storNm',
       'totalCount', 'coordx', 'coordy', 'x', 'y', 'opbizYm', 'clsbizYm',
       '영업일계산기준일', '영업일수'],
      dtype='object')
Index(['상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드',
       '서비스_업종_코드_명', '분기매출금액', '당월_매출_건수', '점포수', '점포당분기매출', '주중_매출_비율',
       '주말_매출_비율', '월요일_매출_비율', '화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
       '금요일_매출_비율', '토요일_매출_비율', '일요일_매출_비율', '시간대_00~06_매출_비율',
       '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
       '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율',
       '연령대_10_매출_비율', '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율',
       '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_

In [22]:
print('점포이력 : ',df_점포이력최종.shape,'  ','추정매출 : ',df_추정매출.shape)

점포이력 :  (1216720, 15)    추정매출 :  (65134, 79)


In [23]:
merge = pd.merge(df_점포이력최종,df_추정매출, on=['상권_코드_명','서비스_업종_코드_명'], how='inner')
merge.shape

(1045184, 92)

In [24]:
idx_업체명 = merge[merge.storNm.isnull()].index
merge.storNm.iloc[idx_업체명] = '없음'
merge.iloc[idx_업체명].storNm

/Users/SSO/opt/anaconda3/envs/crawling/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


274609    없음
408734    없음
664225    없음
695218    없음
859319    없음
860090    없음
863548    없음
Name: storNm, dtype: object

In [25]:
merge.isnull().sum().sum()

0

In [26]:
df_점포 = pd.read_pickle('df_점포.pki')
df_직장인구 = pd.read_pickle('df_직장인구.pki')
df_추정유동인구 = pd.read_pickle('df_추정유동인구.pki')
df_상주인구 = pd.read_pickle('df_상주인구.pki')
df_아파트 = pd.read_pickle('df_아파트.pki')
df_집객시설 = pd.read_pickle('df_집객시설.pki')
df_상권임대시세_소득가구수 = pd.read_pickle('df_상권임대시세_소득가구수.pki')

In [27]:
# merge 1045184, 92
# df_점포 119921 rows × 13 columns
# 콜론 92 + 13 - 6 = 99
# 머신러닝 타겟값인 추정매출 코드 종유 63개에 맞춰서 merge

merge1 = pd.merge(merge,df_점포, on = ['상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명','서비스_업종_코드','서비스_업종_코드_명'], how='inner')
merge1.shape

(1045178, 99)

In [28]:
merge1.isnull().sum().sum()

0

In [29]:
col = ['상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명']

In [30]:
99+25-4

120

In [31]:
# merge1 1045178, 99
# df_직장인구 # 1477 rows × 25 columns
# 공통콜론 4개 99+25-4= 120

merge2 = pd.merge(merge1,df_직장인구, on = col, how='inner')
merge2.shape # 약 5000개 줄었다. 총 상권 갯수 1496 인데 df_직장인구 상권 갯수는 1477 밖에 없음

(1040598, 120)

In [32]:
merge2.isnull().sum().sum()

0

In [33]:
120+530-4

646

In [34]:
# merge2 1040598, 120
# df_추정유동인구 # 1495 rows × 530 columns
# 콜론 120+530-4 = 646

merge3 = pd.merge(merge2,df_추정유동인구, on = col, how='inner')
merge3.shape

(1040598, 646)

In [35]:
merge3.isnull().sum().sum()

0

In [36]:
646+28-4

670

In [37]:
# merge3 1040598, 646
# df_상주인구 # 1482 rows × 28 columns
# 콜론 646+28-4 = 670

merge4 = pd.merge(merge3,df_상주인구, on = col, how='inner')
merge4.shape # 약 만개 날아감 ㅠㅠ

(1032749, 670)

In [38]:
merge4.isnull().sum().sum()

0

In [39]:
670+19-4

685

In [65]:
# merge4 1032749, 670
# df_아파트 # 1341 rows × 19 columns
# 콜론 670+19-4 = 685
# 아파트는 inner 하지 말고 outer 로 가자 .. 아파트 없는 지역은 fillna 로 간다.

merge5 = pd.merge(merge4,df_아파트, on = col, how='outer')
merge5.shape

(1032760, 685)

In [66]:
merge5.isnull().sum().sum()

1957521

In [67]:
# 서비스업종코드 nan 값 11개 전부 제거
idx = merge5[merge5.서비스_업종_코드.isnull()].index
merge5.drop(index = idx, inplace=True)
merge5.shape

(1032749, 685)

In [68]:
merge5.iloc[:,-19:] = merge5.iloc[:,-19:].fillna(0)

In [69]:
merge5.iloc[:,-19:].isnull().sum()

여성연령대_60_이상_상주인구_수        0
총_가구_수                    0
아파트_가구_수                  0
비_아파트_가구_수                0
아파트_단지_수                  0
아파트_면적_66_제곱미터_미만_세대_수    0
아파트_면적_66_제곱미터_세대_수       0
아파트_면적_99_제곱미터_세대_수       0
아파트_면적_132_제곱미터_세대_수      0
아파트_면적_165_제곱미터_세대_수      0
아파트_가격_1_억_미만_세대_수        0
아파트_가격_1_억_세대_수           0
아파트_가격_2_억_세대_수           0
아파트_가격_3_억_세대_수           0
아파트_가격_4_억_세대_수           0
아파트_가격_5_억_세대_수           0
아파트_가격_6_억_이상_세대_수        0
아파트_평균_면적                 0
아파트_평균_시가                 0
dtype: int64

In [70]:
merge5.isnull().sum().sum()

0

In [71]:
merge5.shape

(1032749, 685)

In [73]:
merge5.to_pickle('merge5_작업중20200120.pki')

In [74]:
merge5 = pd.read_pickle('merge5_작업중20200120.pki')
merge5.shape

(1032749, 685)

In [75]:
685 + 24 - 4

705

In [76]:
# merge5 1032749, 685
# df_집객시설 # 1464 rows × 24 columns
# 콜론 685 + 24 - 4 = 705

merge6 = pd.merge(merge5,df_집객시설, on = col, how='outer')
merge6.shape

(1032775, 705)

In [77]:
# 서비스_업종_코드_명 nan 값은 제거
idx = merge6[merge6.서비스_업종_코드.isnull()].index
merge6.drop(index = idx, inplace=True)
merge6.shape

(1032749, 705)

In [78]:
merge6.iloc[:,-20:] = merge6.iloc[:,-20:].fillna(0)
merge6.isnull().sum().sum()

0

In [80]:
705 + 13 - 4

714

In [81]:
# merge6 1032749, 705
# df_상권임대시세_소득가구수 # 1496 rows × 13 columns
# 콜론 705 + 13 - 4 = 714

merge7 = pd.merge(merge6,df_상권임대시세_소득가구수, on = col, how='inner')
merge7.shape

(1032749, 714)

In [83]:
merge7.isnull().sum().sum()

0

In [85]:
col_lis = []
m7_col = merge7.columns
for i in m7_col:
    col_lis.append(i)
col_lis

['adres',
 '도로명',
 '상권_코드_명',
 '서비스_업종_코드_명',
 '소분류업종',
 'storNm',
 'totalCount',
 'coordx',
 'coordy',
 'x',
 'y',
 'opbizYm',
 'clsbizYm',
 '영업일계산기준일',
 '영업일수',
 '상권_구분_코드',
 '상권_구분_코드_명',
 '상권_코드',
 '서비스_업종_코드',
 '분기매출금액',
 '당월_매출_건수',
 '점포수',
 '점포당분기매출',
 '주중_매출_비율',
 '주말_매출_비율',
 '월요일_매출_비율',
 '화요일_매출_비율',
 '수요일_매출_비율',
 '목요일_매출_비율',
 '금요일_매출_비율',
 '토요일_매출_비율',
 '일요일_매출_비율',
 '시간대_00~06_매출_비율',
 '시간대_06~11_매출_비율',
 '시간대_11~14_매출_비율',
 '시간대_14~17_매출_비율',
 '시간대_17~21_매출_비율',
 '시간대_21~24_매출_비율',
 '남성_매출_비율',
 '여성_매출_비율',
 '연령대_10_매출_비율',
 '연령대_20_매출_비율',
 '연령대_30_매출_비율',
 '연령대_40_매출_비율',
 '연령대_50_매출_비율',
 '연령대_60_이상_매출_비율',
 '주중_매출_금액',
 '주말_매출_금액',
 '월요일_매출_금액',
 '화요일_매출_금액',
 '수요일_매출_금액',
 '목요일_매출_금액',
 '금요일_매출_금액',
 '토요일_매출_금액',
 '일요일_매출_금액',
 '시간대_00~06_매출_금액',
 '시간대_06~11_매출_금액',
 '시간대_11~14_매출_금액',
 '시간대_14~17_매출_금액',
 '시간대_17~21_매출_금액',
 '시간대_21~24_매출_금액',
 '남성_매출_금액',
 '여성_매출_금액',
 '연령대_10_매출_금액',
 '연령대_20_매출_금액',
 '연령대_30_매출_금액',
 '연령대_40_매출_금액',
 '연령대_50_매출_금액',
 '연령대_60_이상

In [87]:
# columns 정렬
merge7_콜론정리 = merge7[[
            'adres',
         '도로명',
         '상권_코드_명',
         '서비스_업종_코드_명',
         '소분류업종',
         'storNm',
         'totalCount',
         'coordx',
         'coordy',
         'x',
         'y',
         'opbizYm',
         'clsbizYm',
         '영업일계산기준일',
         '상권_구분_코드',
         '상권_구분_코드_명',
         '상권_코드',
         '서비스_업종_코드',
         '분기매출금액',
         '당월_매출_건수',
         '엑스좌표_값',
         '와이좌표_값',
         '행정동_코드_명',
         '임대시세_1층',
         '임대시세_1층외',
         '소득분위', 
         '점포당분기매출',
         '임대시세_전체',
         '영업일수',
         '가구수',
         '소득분위점수',
         '점포수',
         '주중_매출_비율',
         '주말_매출_비율',
         '월요일_매출_비율',
         '화요일_매출_비율',
         '수요일_매출_비율',
         '목요일_매출_비율',
         '금요일_매출_비율',
         '토요일_매출_비율',
         '일요일_매출_비율',
         '시간대_00~06_매출_비율',
         '시간대_06~11_매출_비율',
         '시간대_11~14_매출_비율',
         '시간대_14~17_매출_비율',
         '시간대_17~21_매출_비율',
         '시간대_21~24_매출_비율',
         '남성_매출_비율',
         '여성_매출_비율',
         '연령대_10_매출_비율',
         '연령대_20_매출_비율',
         '연령대_30_매출_비율',
         '연령대_40_매출_비율',
         '연령대_50_매출_비율',
         '연령대_60_이상_매출_비율',
         '주중_매출_금액',
         '주말_매출_금액',
         '월요일_매출_금액',
         '화요일_매출_금액',
         '수요일_매출_금액',
         '목요일_매출_금액',
         '금요일_매출_금액',
         '토요일_매출_금액',
         '일요일_매출_금액',
         '시간대_00~06_매출_금액',
         '시간대_06~11_매출_금액',
         '시간대_11~14_매출_금액',
         '시간대_14~17_매출_금액',
         '시간대_17~21_매출_금액',
         '시간대_21~24_매출_금액',
         '남성_매출_금액',
         '여성_매출_금액',
         '연령대_10_매출_금액',
         '연령대_20_매출_금액',
         '연령대_30_매출_금액',
         '연령대_40_매출_금액',
         '연령대_50_매출_금액',
         '연령대_60_이상_매출_금액',
         '주중_매출_건수',
         '주말_매출_건수',
         '월요일_매출_건수',
         '화요일_매출_건수',
         '수요일_매출_건수',
         '목요일_매출_건수',
         '금요일_매출_건수',
         '토요일_매출_건수',
         '일요일_매출_건수',
         '시간대_건수~06_매출_건수',
         '시간대_건수~11_매출_건수',
         '시간대_건수~14_매출_건수',
         '시간대_건수~17_매출_건수',
         '시간대_건수~21_매출_건수',
         '시간대_건수~24_매출_건수',
         '남성_매출_건수',
         '여성_매출_건수',
         '연령대_10_매출_건수',
         '연령대_20_매출_건수',
         '연령대_30_매출_건수',
         '연령대_40_매출_건수',
         '연령대_50_매출_건수',
         '연령대_60_이상_매출_건수',
         '점포_수',
         '유사_업종_점포_수',
         '개업_율',
         '개업_점포_수',
         '폐업_률',
         '폐업_점포_수',
         '프랜차이즈_점포_수',
         '총_직장_인구_수',
         '남성_직장_인구_수',
         '여성_직장_인구_수',
         '연령대_10_직장_인구_수',
         '연령대_20_직장_인구_수',
         '연령대_30_직장_인구_수',
         '연령대_40_직장_인구_수',
         '연령대_50_직장_인구_수',
         '연령대_60_이상_직장_인구_수',
         '남성연령대_10_직장_인구_수',
         '남성연령대_20_직장_인구_수',
         '남성연령대_30_직장_인구_수',
         '남성연령대_40_직장_인구_수',
         '남성연령대_50_직장_인구_수',
         '남성연령대_60_이상_직장_인구_수',
         '여성연령대_10_직장_인구_수',
         '여성연령대_20_직장_인구_수',
         '여성연령대_30_직장_인구_수',
         '여성연령대_40_직장_인구_수',
         '여성연령대_50_직장_인구_수',
         '여성연령대_60_이상_직장_인구_수',
         '총_유동인구_수',
         '남성_유동인구_수',
         '여성_유동인구_수',
         '연령대_10_유동인구_수',
         '연령대_20_유동인구_수',
         '연령대_30_유동인구_수',
         '연령대_40_유동인구_수',
         '연령대_50_유동인구_수',
         '연령대_60_이상_유동인구_수',
         '시간대_1_유동인구_수',
         '시간대_2_유동인구_수',
         '시간대_3_유동인구_수',
         '시간대_4_유동인구_수',
         '시간대_5_유동인구_수',
         '시간대_6_유동인구_수',
         '월요일_유동인구_수',
         '화요일_유동인구_수',
         '수요일_유동인구_수',
         '목요일_유동인구_수',
         '금요일_유동인구_수',
         '토요일_유동인구_수',
         '일요일_유동인구_수',
         '남성연령대_10_월요일시간대_1_유동인구_수',
         '남성연령대_10_월요일시간대_2_유동인구_수',
         '남성연령대_10_월요일시간대_3_유동인구_수',
         '남성연령대_10_월요일시간대_4_유동인구_수',
         '남성연령대_10_월요일시간대_5_유동인구_수',
         '남성연령대_10_월요일시간대_6_유동인구_수',
         '남성연령대_10_화요일시간대_1_유동인구_수',
         '남성연령대_10_화요일시간대_2_유동인구_수',
         '남성연령대_10_화요일시간대_3_유동인구_수',
         '남성연령대_10_화요일시간대_4_유동인구_수',
         '남성연령대_10_화요일시간대_5_유동인구_수',
         '남성연령대_10_화요일시간대_6_유동인구_수',
         '남성연령대_10_수요일시간대_1_유동인구_수',
         '남성연령대_10_수요일시간대_2_유동인구_수',
         '남성연령대_10_수요일시간대_3_유동인구_수',
         '남성연령대_10_수요일시간대_4_유동인구_수',
         '남성연령대_10_수요일시간대_5_유동인구_수',
         '남성연령대_10_수요일시간대_6_유동인구_수',
         '남성연령대_10_목요일시간대_1_유동인구_수',
         '남성연령대_10_목요일시간대_2_유동인구_수',
         '남성연령대_10_목요일시간대_3_유동인구_수',
         '남성연령대_10_목요일시간대_4_유동인구_수',
         '남성연령대_10_목요일시간대_5_유동인구_수',
         '남성연령대_10_목요일시간대_6_유동인구_수',
         '남성연령대_10_금요일시간대_1_유동인구_수',
         '남성연령대_10_금요일시간대_2_유동인구_수',
         '남성연령대_10_금요일시간대_3_유동인구_수',
         '남성연령대_10_금요일시간대_4_유동인구_수',
         '남성연령대_10_금요일시간대_5_유동인구_수',
         '남성연령대_10_금요일시간대_6_유동인구_수',
         '남성연령대_10_토요일시간대_1_유동인구_수',
         '남성연령대_10_토요일시간대_2_유동인구_수',
         '남성연령대_10_토요일시간대_3_유동인구_수',
         '남성연령대_10_토요일시간대_4_유동인구_수',
         '남성연령대_10_토요일시간대_5_유동인구_수',
         '남성연령대_10_토요일시간대_6_유동인구_수',
         '남성연령대_10_일요일시간대_1_유동인구_수',
         '남성연령대_10_일요일시간대_2_유동인구_수',
         '남성연령대_10_일요일시간대_3_유동인구_수',
         '남성연령대_10_일요일시간대_4_유동인구_수',
         '남성연령대_10_일요일시간대_5_유동인구_수',
         '남성연령대_10_일요일시간대_6_유동인구_수',
         '남성연령대_20_월요일시간대_1_유동인구_수',
         '남성연령대_20_월요일시간대_2_유동인구_수',
         '남성연령대_20_월요일시간대_3_유동인구_수',
         '남성연령대_20_월요일시간대_4_유동인구_수',
         '남성연령대_20_월요일시간대_5_유동인구_수',
         '남성연령대_20_월요일시간대_6_유동인구_수',
         '남성연령대_20_화요일시간대_1_유동인구_수',
         '남성연령대_20_화요일시간대_2_유동인구_수',
         '남성연령대_20_화요일시간대_3_유동인구_수',
         '남성연령대_20_화요일시간대_4_유동인구_수',
         '남성연령대_20_화요일시간대_5_유동인구_수',
         '남성연령대_20_화요일시간대_6_유동인구_수',
         '남성연령대_20_수요일시간대_1_유동인구_수',
         '남성연령대_20_수요일시간대_2_유동인구_수',
         '남성연령대_20_수요일시간대_3_유동인구_수',
         '남성연령대_20_수요일시간대_4_유동인구_수',
         '남성연령대_20_수요일시간대_5_유동인구_수',
         '남성연령대_20_수요일시간대_6_유동인구_수',
         '남성연령대_20_목요일시간대_1_유동인구_수',
         '남성연령대_20_목요일시간대_2_유동인구_수',
         '남성연령대_20_목요일시간대_3_유동인구_수',
         '남성연령대_20_목요일시간대_4_유동인구_수',
         '남성연령대_20_목요일시간대_5_유동인구_수',
         '남성연령대_20_목요일시간대_6_유동인구_수',
         '남성연령대_20_금요일시간대_1_유동인구_수',
         '남성연령대_20_금요일시간대_2_유동인구_수',
         '남성연령대_20_금요일시간대_3_유동인구_수',
         '남성연령대_20_금요일시간대_4_유동인구_수',
         '남성연령대_20_금요일시간대_5_유동인구_수',
         '남성연령대_20_금요일시간대_6_유동인구_수',
         '남성연령대_20_토요일시간대_1_유동인구_수',
         '남성연령대_20_토요일시간대_2_유동인구_수',
         '남성연령대_20_토요일시간대_3_유동인구_수',
         '남성연령대_20_토요일시간대_4_유동인구_수',
         '남성연령대_20_토요일시간대_5_유동인구_수',
         '남성연령대_20_토요일시간대_6_유동인구_수',
         '남성연령대_20_일요일시간대_1_유동인구_수',
         '남성연령대_20_일요일시간대_2_유동인구_수',
         '남성연령대_20_일요일시간대_3_유동인구_수',
         '남성연령대_20_일요일시간대_4_유동인구_수',
         '남성연령대_20_일요일시간대_5_유동인구_수',
         '남성연령대_20_일요일시간대_6_유동인구_수',
         '남성연령대_30_월요일시간대_1_유동인구_수',
         '남성연령대_30_월요일시간대_2_유동인구_수',
         '남성연령대_30_월요일시간대_3_유동인구_수',
         '남성연령대_30_월요일시간대_4_유동인구_수',
         '남성연령대_30_월요일시간대_5_유동인구_수',
         '남성연령대_30_월요일시간대_6_유동인구_수',
         '남성연령대_30_화요일시간대_1_유동인구_수',
         '남성연령대_30_화요일시간대_2_유동인구_수',
         '남성연령대_30_화요일시간대_3_유동인구_수',
         '남성연령대_30_화요일시간대_4_유동인구_수',
         '남성연령대_30_화요일시간대_5_유동인구_수',
         '남성연령대_30_화요일시간대_6_유동인구_수',
         '남성연령대_30_수요일시간대_1_유동인구_수',
         '남성연령대_30_수요일시간대_2_유동인구_수',
         '남성연령대_30_수요일시간대_3_유동인구_수',
         '남성연령대_30_수요일시간대_4_유동인구_수',
         '남성연령대_30_수요일시간대_5_유동인구_수',
         '남성연령대_30_수요일시간대_6_유동인구_수',
         '남성연령대_30_목요일시간대_1_유동인구_수',
         '남성연령대_30_목요일시간대_2_유동인구_수',
         '남성연령대_30_목요일시간대_3_유동인구_수',
         '남성연령대_30_목요일시간대_4_유동인구_수',
         '남성연령대_30_목요일시간대_5_유동인구_수',
         '남성연령대_30_목요일시간대_6_유동인구_수',
         '남성연령대_30_금요일시간대_1_유동인구_수',
         '남성연령대_30_금요일시간대_2_유동인구_수',
         '남성연령대_30_금요일시간대_3_유동인구_수',
         '남성연령대_30_금요일시간대_4_유동인구_수',
         '남성연령대_30_금요일시간대_5_유동인구_수',
         '남성연령대_30_금요일시간대_6_유동인구_수',
         '남성연령대_30_토요일시간대_1_유동인구_수',
         '남성연령대_30_토요일시간대_2_유동인구_수',
         '남성연령대_30_토요일시간대_3_유동인구_수',
         '남성연령대_30_토요일시간대_4_유동인구_수',
         '남성연령대_30_토요일시간대_5_유동인구_수',
         '남성연령대_30_토요일시간대_6_유동인구_수',
         '남성연령대_30_일요일시간대_1_유동인구_수',
         '남성연령대_30_일요일시간대_2_유동인구_수',
         '남성연령대_30_일요일시간대_3_유동인구_수',
         '남성연령대_30_일요일시간대_4_유동인구_수',
         '남성연령대_30_일요일시간대_5_유동인구_수',
         '남성연령대_30_일요일시간대_6_유동인구_수',
         '남성연령대_40_월요일시간대_1_유동인구_수',
         '남성연령대_40_월요일시간대_2_유동인구_수',
         '남성연령대_40_월요일시간대_3_유동인구_수',
         '남성연령대_40_월요일시간대_4_유동인구_수',
         '남성연령대_40_월요일시간대_5_유동인구_수',
         '남성연령대_40_월요일시간대_6_유동인구_수',
         '남성연령대_40_화요일시간대_1_유동인구_수',
         '남성연령대_40_화요일시간대_2_유동인구_수',
         '남성연령대_40_화요일시간대_3_유동인구_수',
         '남성연령대_40_화요일시간대_4_유동인구_수',
         '남성연령대_40_화요일시간대_5_유동인구_수',
         '남성연령대_40_화요일시간대_6_유동인구_수',
         '남성연령대_40_수요일시간대_1_유동인구_수',
         '남성연령대_40_수요일시간대_2_유동인구_수',
         '남성연령대_40_수요일시간대_3_유동인구_수',
         '남성연령대_40_수요일시간대_4_유동인구_수',
         '남성연령대_40_수요일시간대_5_유동인구_수',
         '남성연령대_40_수요일시간대_6_유동인구_수',
         '남성연령대_40_목요일시간대_1_유동인구_수',
         '남성연령대_40_목요일시간대_2_유동인구_수',
         '남성연령대_40_목요일시간대_3_유동인구_수',
         '남성연령대_40_목요일시간대_4_유동인구_수',
         '남성연령대_40_목요일시간대_5_유동인구_수',
         '남성연령대_40_목요일시간대_6_유동인구_수',
         '남성연령대_40_금요일시간대_1_유동인구_수',
         '남성연령대_40_금요일시간대_2_유동인구_수',
         '남성연령대_40_금요일시간대_3_유동인구_수',
         '남성연령대_40_금요일시간대_4_유동인구_수',
         '남성연령대_40_금요일시간대_5_유동인구_수',
         '남성연령대_40_금요일시간대_6_유동인구_수',
         '남성연령대_40_토요일시간대_1_유동인구_수',
         '남성연령대_40_토요일시간대_2_유동인구_수',
         '남성연령대_40_토요일시간대_3_유동인구_수',
         '남성연령대_40_토요일시간대_4_유동인구_수',
         '남성연령대_40_토요일시간대_5_유동인구_수',
         '남성연령대_40_토요일시간대_6_유동인구_수',
         '남성연령대_40_일요일시간대_1_유동인구_수',
         '남성연령대_40_일요일시간대_2_유동인구_수',
         '남성연령대_40_일요일시간대_3_유동인구_수',
         '남성연령대_40_일요일시간대_4_유동인구_수',
         '남성연령대_40_일요일시간대_5_유동인구_수',
         '남성연령대_40_일요일시간대_6_유동인구_수',
         '남성연령대_50_월요일시간대_1_유동인구_수',
         '남성연령대_50_월요일시간대_2_유동인구_수',
         '남성연령대_50_월요일시간대_3_유동인구_수',
         '남성연령대_50_월요일시간대_4_유동인구_수',
         '남성연령대_50_월요일시간대_5_유동인구_수',
         '남성연령대_50_월요일시간대_6_유동인구_수',
         '남성연령대_50_화요일시간대_1_유동인구_수',
         '남성연령대_50_화요일시간대_2_유동인구_수',
         '남성연령대_50_화요일시간대_3_유동인구_수',
         '남성연령대_50_화요일시간대_4_유동인구_수',
         '남성연령대_50_화요일시간대_5_유동인구_수',
         '남성연령대_50_화요일시간대_6_유동인구_수',
         '남성연령대_50_수요일시간대_1_유동인구_수',
         '남성연령대_50_수요일시간대_2_유동인구_수',
         '남성연령대_50_수요일시간대_3_유동인구_수',
         '남성연령대_50_수요일시간대_4_유동인구_수',
         '남성연령대_50_수요일시간대_5_유동인구_수',
         '남성연령대_50_수요일시간대_6_유동인구_수',
         '남성연령대_50_목요일시간대_1_유동인구_수',
         '남성연령대_50_목요일시간대_2_유동인구_수',
         '남성연령대_50_목요일시간대_3_유동인구_수',
         '남성연령대_50_목요일시간대_4_유동인구_수',
         '남성연령대_50_목요일시간대_5_유동인구_수',
         '남성연령대_50_목요일시간대_6_유동인구_수',
         '남성연령대_50_금요일시간대_1_유동인구_수',
         '남성연령대_50_금요일시간대_2_유동인구_수',
         '남성연령대_50_금요일시간대_3_유동인구_수',
         '남성연령대_50_금요일시간대_4_유동인구_수',
         '남성연령대_50_금요일시간대_5_유동인구_수',
         '남성연령대_50_금요일시간대_6_유동인구_수',
         '남성연령대_50_토요일시간대_1_유동인구_수',
         '남성연령대_50_토요일시간대_2_유동인구_수',
         '남성연령대_50_토요일시간대_3_유동인구_수',
         '남성연령대_50_토요일시간대_4_유동인구_수',
         '남성연령대_50_토요일시간대_5_유동인구_수',
         '남성연령대_50_토요일시간대_6_유동인구_수',
         '남성연령대_50_일요일시간대_1_유동인구_수',
         '남성연령대_50_일요일시간대_2_유동인구_수',
         '남성연령대_50_일요일시간대_3_유동인구_수',
         '남성연령대_50_일요일시간대_4_유동인구_수',
         '남성연령대_50_일요일시간대_5_유동인구_수',
         '남성연령대_50_일요일시간대_6_유동인구_수',
         '남성연령대_60_이상_월요일시간대_1_유동인구_수',
         '남성연령대_60_이상_월요일시간대_2_유동인구_수',
         '남성연령대_60_이상_월요일시간대_3_유동인구_수',
         '남성연령대_60_이상_월요일시간대_4_유동인구_수',
         '남성연령대_60_이상_월요일시간대_5_유동인구_수',
         '남성연령대_60_이상_월요일시간대_6_유동인구_수',
         '남성연령대_60_이상_화요일시간대_1_유동인구_수',
         '남성연령대_60_이상_화요일시간대_2_유동인구_수',
         '남성연령대_60_이상_화요일시간대_3_유동인구_수',
         '남성연령대_60_이상_화요일시간대_4_유동인구_수',
         '남성연령대_60_이상_화요일시간대_5_유동인구_수',
         '남성연령대_60_이상_화요일시간대_6_유동인구_수',
         '남성연령대_60_이상_수요일시간대_1_유동인구_수',
         '남성연령대_60_이상_수요일시간대_2_유동인구_수',
         '남성연령대_60_이상_수요일시간대_3_유동인구_수',
         '남성연령대_60_이상_수요일시간대_4_유동인구_수',
         '남성연령대_60_이상_수요일시간대_5_유동인구_수',
         '남성연령대_60_이상_수요일시간대_6_유동인구_수',
         '남성연령대_60_이상_목요일시간대_1_유동인구_수',
         '남성연령대_60_이상_목요일시간대_2_유동인구_수',
         '남성연령대_60_이상_목요일시간대_3_유동인구_수',
         '남성연령대_60_이상_목요일시간대_4_유동인구_수',
         '남성연령대_60_이상_목요일시간대_5_유동인구_수',
         '남성연령대_60_이상_목요일시간대_6_유동인구_수',
         '남성연령대_60_이상_금요일시간대_1_유동인구_수',
         '남성연령대_60_이상_금요일시간대_2_유동인구_수',
         '남성연령대_60_이상_금요일시간대_3_유동인구_수',
         '남성연령대_60_이상_금요일시간대_4_유동인구_수',
         '남성연령대_60_이상_금요일시간대_5_유동인구_수',
         '남성연령대_60_이상_금요일시간대_6_유동인구_수',
         '남성연령대_60_이상_토요일시간대_1_유동인구_수',
         '남성연령대_60_이상_토요일시간대_2_유동인구_수',
         '남성연령대_60_이상_토요일시간대_3_유동인구_수',
         '남성연령대_60_이상_토요일시간대_4_유동인구_수',
         '남성연령대_60_이상_토요일시간대_5_유동인구_수',
         '남성연령대_60_이상_토요일시간대_6_유동인구_수',
         '남성연령대_60_이상_일요일시간대_1_유동인구_수',
         '남성연령대_60_이상_일요일시간대_2_유동인구_수',
         '남성연령대_60_이상_일요일시간대_3_유동인구_수',
         '남성연령대_60_이상_일요일시간대_4_유동인구_수',
         '남성연령대_60_이상_일요일시간대_5_유동인구_수',
         '남성연령대_60_이상_일요일시간대_6_유동인구_수',
         '여성연령대_10_월요일시간대_1_유동인구_수',
         '여성연령대_10_월요일시간대_2_유동인구_수',
         '여성연령대_10_월요일시간대_3_유동인구_수',
         '여성연령대_10_월요일시간대_4_유동인구_수',
         '여성연령대_10_월요일시간대_5_유동인구_수',
         '여성연령대_10_월요일시간대_6_유동인구_수',
         '여성연령대_10_화요일시간대_1_유동인구_수',
         '여성연령대_10_화요일시간대_2_유동인구_수',
         '여성연령대_10_화요일시간대_3_유동인구_수',
         '여성연령대_10_화요일시간대_4_유동인구_수',
         '여성연령대_10_화요일시간대_5_유동인구_수',
         '여성연령대_10_화요일시간대_6_유동인구_수',
         '여성연령대_10_수요일시간대_1_유동인구_수',
         '여성연령대_10_수요일시간대_2_유동인구_수',
         '여성연령대_10_수요일시간대_3_유동인구_수',
         '여성연령대_10_수요일시간대_4_유동인구_수',
         '여성연령대_10_수요일시간대_5_유동인구_수',
         '여성연령대_10_수요일시간대_6_유동인구_수',
         '여성연령대_10_목요일시간대_1_유동인구_수',
         '여성연령대_10_목요일시간대_2_유동인구_수',
         '여성연령대_10_목요일시간대_3_유동인구_수',
         '여성연령대_10_목요일시간대_4_유동인구_수',
         '여성연령대_10_목요일시간대_5_유동인구_수',
         '여성연령대_10_목요일시간대_6_유동인구_수',
         '여성연령대_10_금요일시간대_1_유동인구_수',
         '여성연령대_10_금요일시간대_2_유동인구_수',
         '여성연령대_10_금요일시간대_3_유동인구_수',
         '여성연령대_10_금요일시간대_4_유동인구_수',
         '여성연령대_10_금요일시간대_5_유동인구_수',
         '여성연령대_10_금요일시간대_6_유동인구_수',
         '여성연령대_10_토요일시간대_1_유동인구_수',
         '여성연령대_10_토요일시간대_2_유동인구_수',
         '여성연령대_10_토요일시간대_3_유동인구_수',
         '여성연령대_10_토요일시간대_4_유동인구_수',
         '여성연령대_10_토요일시간대_5_유동인구_수',
         '여성연령대_10_토요일시간대_6_유동인구_수',
         '여성연령대_10_일요일시간대_1_유동인구_수',
         '여성연령대_10_일요일시간대_2_유동인구_수',
         '여성연령대_10_일요일시간대_3_유동인구_수',
         '여성연령대_10_일요일시간대_4_유동인구_수',
         '여성연령대_10_일요일시간대_5_유동인구_수',
         '여성연령대_10_일요일시간대_6_유동인구_수',
         '여성연령대_20_월요일시간대_1_유동인구_수',
         '여성연령대_20_월요일시간대_2_유동인구_수',
         '여성연령대_20_월요일시간대_3_유동인구_수',
         '여성연령대_20_월요일시간대_4_유동인구_수',
         '여성연령대_20_월요일시간대_5_유동인구_수',
         '여성연령대_20_월요일시간대_6_유동인구_수',
         '여성연령대_20_화요일시간대_1_유동인구_수',
         '여성연령대_20_화요일시간대_2_유동인구_수',
         '여성연령대_20_화요일시간대_3_유동인구_수',
         '여성연령대_20_화요일시간대_4_유동인구_수',
         '여성연령대_20_화요일시간대_5_유동인구_수',
         '여성연령대_20_화요일시간대_6_유동인구_수',
         '여성연령대_20_수요일시간대_1_유동인구_수',
         '여성연령대_20_수요일시간대_2_유동인구_수',
         '여성연령대_20_수요일시간대_3_유동인구_수',
         '여성연령대_20_수요일시간대_4_유동인구_수',
         '여성연령대_20_수요일시간대_5_유동인구_수',
         '여성연령대_20_수요일시간대_6_유동인구_수',
         '여성연령대_20_목요일시간대_1_유동인구_수',
         '여성연령대_20_목요일시간대_2_유동인구_수',
         '여성연령대_20_목요일시간대_3_유동인구_수',
         '여성연령대_20_목요일시간대_4_유동인구_수',
         '여성연령대_20_목요일시간대_5_유동인구_수',
         '여성연령대_20_목요일시간대_6_유동인구_수',
         '여성연령대_20_금요일시간대_1_유동인구_수',
         '여성연령대_20_금요일시간대_2_유동인구_수',
         '여성연령대_20_금요일시간대_3_유동인구_수',
         '여성연령대_20_금요일시간대_4_유동인구_수',
         '여성연령대_20_금요일시간대_5_유동인구_수',
         '여성연령대_20_금요일시간대_6_유동인구_수',
         '여성연령대_20_토요일시간대_1_유동인구_수',
         '여성연령대_20_토요일시간대_2_유동인구_수',
         '여성연령대_20_토요일시간대_3_유동인구_수',
         '여성연령대_20_토요일시간대_4_유동인구_수',
         '여성연령대_20_토요일시간대_5_유동인구_수',
         '여성연령대_20_토요일시간대_6_유동인구_수',
         '여성연령대_20_일요일시간대_1_유동인구_수',
         '여성연령대_20_일요일시간대_2_유동인구_수',
         '여성연령대_20_일요일시간대_3_유동인구_수',
         '여성연령대_20_일요일시간대_4_유동인구_수',
         '여성연령대_20_일요일시간대_5_유동인구_수',
         '여성연령대_20_일요일시간대_6_유동인구_수',
         '여성연령대_30_월요일시간대_1_유동인구_수',
         '여성연령대_30_월요일시간대_2_유동인구_수',
         '여성연령대_30_월요일시간대_3_유동인구_수',
         '여성연령대_30_월요일시간대_4_유동인구_수',
         '여성연령대_30_월요일시간대_5_유동인구_수',
         '여성연령대_30_월요일시간대_6_유동인구_수',
         '여성연령대_30_화요일시간대_1_유동인구_수',
         '여성연령대_30_화요일시간대_2_유동인구_수',
         '여성연령대_30_화요일시간대_3_유동인구_수',
         '여성연령대_30_화요일시간대_4_유동인구_수',
         '여성연령대_30_화요일시간대_5_유동인구_수',
         '여성연령대_30_화요일시간대_6_유동인구_수',
         '여성연령대_30_수요일시간대_1_유동인구_수',
         '여성연령대_30_수요일시간대_2_유동인구_수',
         '여성연령대_30_수요일시간대_3_유동인구_수',
         '여성연령대_30_수요일시간대_4_유동인구_수',
         '여성연령대_30_수요일시간대_5_유동인구_수',
         '여성연령대_30_수요일시간대_6_유동인구_수',
         '여성연령대_30_목요일시간대_1_유동인구_수',
         '여성연령대_30_목요일시간대_2_유동인구_수',
         '여성연령대_30_목요일시간대_3_유동인구_수',
         '여성연령대_30_목요일시간대_4_유동인구_수',
         '여성연령대_30_목요일시간대_5_유동인구_수',
         '여성연령대_30_목요일시간대_6_유동인구_수',
         '여성연령대_30_금요일시간대_1_유동인구_수',
         '여성연령대_30_금요일시간대_2_유동인구_수',
         '여성연령대_30_금요일시간대_3_유동인구_수',
         '여성연령대_30_금요일시간대_4_유동인구_수',
         '여성연령대_30_금요일시간대_5_유동인구_수',
         '여성연령대_30_금요일시간대_6_유동인구_수',
         '여성연령대_30_토요일시간대_1_유동인구_수',
         '여성연령대_30_토요일시간대_2_유동인구_수',
         '여성연령대_30_토요일시간대_3_유동인구_수',
         '여성연령대_30_토요일시간대_4_유동인구_수',
         '여성연령대_30_토요일시간대_5_유동인구_수',
         '여성연령대_30_토요일시간대_6_유동인구_수',
         '여성연령대_30_일요일시간대_1_유동인구_수',
         '여성연령대_30_일요일시간대_2_유동인구_수',
         '여성연령대_30_일요일시간대_3_유동인구_수',
         '여성연령대_30_일요일시간대_4_유동인구_수',
         '여성연령대_30_일요일시간대_5_유동인구_수',
         '여성연령대_30_일요일시간대_6_유동인구_수',
         '여성연령대_40_월요일시간대_1_유동인구_수',
         '여성연령대_40_월요일시간대_2_유동인구_수',
         '여성연령대_40_월요일시간대_3_유동인구_수',
         '여성연령대_40_월요일시간대_4_유동인구_수',
         '여성연령대_40_월요일시간대_5_유동인구_수',
         '여성연령대_40_월요일시간대_6_유동인구_수',
         '여성연령대_40_화요일시간대_1_유동인구_수',
         '여성연령대_40_화요일시간대_2_유동인구_수',
         '여성연령대_40_화요일시간대_3_유동인구_수',
         '여성연령대_40_화요일시간대_4_유동인구_수',
         '여성연령대_40_화요일시간대_5_유동인구_수',
         '여성연령대_40_화요일시간대_6_유동인구_수',
         '여성연령대_40_수요일시간대_1_유동인구_수',
         '여성연령대_40_수요일시간대_2_유동인구_수',
         '여성연령대_40_수요일시간대_3_유동인구_수',
         '여성연령대_40_수요일시간대_4_유동인구_수',
         '여성연령대_40_수요일시간대_5_유동인구_수',
         '여성연령대_40_수요일시간대_6_유동인구_수',
         '여성연령대_40_목요일시간대_1_유동인구_수',
         '여성연령대_40_목요일시간대_2_유동인구_수',
         '여성연령대_40_목요일시간대_3_유동인구_수',
         '여성연령대_40_목요일시간대_4_유동인구_수',
         '여성연령대_40_목요일시간대_5_유동인구_수',
         '여성연령대_40_목요일시간대_6_유동인구_수',
         '여성연령대_40_금요일시간대_1_유동인구_수',
         '여성연령대_40_금요일시간대_2_유동인구_수',
         '여성연령대_40_금요일시간대_3_유동인구_수',
         '여성연령대_40_금요일시간대_4_유동인구_수',
         '여성연령대_40_금요일시간대_5_유동인구_수',
         '여성연령대_40_금요일시간대_6_유동인구_수',
         '여성연령대_40_토요일시간대_1_유동인구_수',
         '여성연령대_40_토요일시간대_2_유동인구_수',
         '여성연령대_40_토요일시간대_3_유동인구_수',
         '여성연령대_40_토요일시간대_4_유동인구_수',
         '여성연령대_40_토요일시간대_5_유동인구_수',
         '여성연령대_40_토요일시간대_6_유동인구_수',
         '여성연령대_40_일요일시간대_1_유동인구_수',
         '여성연령대_40_일요일시간대_2_유동인구_수',
         '여성연령대_40_일요일시간대_3_유동인구_수',
         '여성연령대_40_일요일시간대_4_유동인구_수',
         '여성연령대_40_일요일시간대_5_유동인구_수',
         '여성연령대_40_일요일시간대_6_유동인구_수',
         '여성연령대_50_월요일시간대_1_유동인구_수',
         '여성연령대_50_월요일시간대_2_유동인구_수',
         '여성연령대_50_월요일시간대_3_유동인구_수',
         '여성연령대_50_월요일시간대_4_유동인구_수',
         '여성연령대_50_월요일시간대_5_유동인구_수',
         '여성연령대_50_월요일시간대_6_유동인구_수',
         '여성연령대_50_화요일시간대_1_유동인구_수',
         '여성연령대_50_화요일시간대_2_유동인구_수',
         '여성연령대_50_화요일시간대_3_유동인구_수',
         '여성연령대_50_화요일시간대_4_유동인구_수',
         '여성연령대_50_화요일시간대_5_유동인구_수',
         '여성연령대_50_화요일시간대_6_유동인구_수',
         '여성연령대_50_수요일시간대_1_유동인구_수',
         '여성연령대_50_수요일시간대_2_유동인구_수',
         '여성연령대_50_수요일시간대_3_유동인구_수',
         '여성연령대_50_수요일시간대_4_유동인구_수',
         '여성연령대_50_수요일시간대_5_유동인구_수',
         '여성연령대_50_수요일시간대_6_유동인구_수',
         '여성연령대_50_목요일시간대_1_유동인구_수',
         '여성연령대_50_목요일시간대_2_유동인구_수',
         '여성연령대_50_목요일시간대_3_유동인구_수',
         '여성연령대_50_목요일시간대_4_유동인구_수',
         '여성연령대_50_목요일시간대_5_유동인구_수',
         '여성연령대_50_목요일시간대_6_유동인구_수',
         '여성연령대_50_금요일시간대_1_유동인구_수',
         '여성연령대_50_금요일시간대_2_유동인구_수',
         '여성연령대_50_금요일시간대_3_유동인구_수',
         '여성연령대_50_금요일시간대_4_유동인구_수',
         '여성연령대_50_금요일시간대_5_유동인구_수',
         '여성연령대_50_금요일시간대_6_유동인구_수',
         '여성연령대_50_토요일시간대_1_유동인구_수',
         '여성연령대_50_토요일시간대_2_유동인구_수',
         '여성연령대_50_토요일시간대_3_유동인구_수',
         '여성연령대_50_토요일시간대_4_유동인구_수',
         '여성연령대_50_토요일시간대_5_유동인구_수',
         '여성연령대_50_토요일시간대_6_유동인구_수',
         '여성연령대_50_일요일시간대_1_유동인구_수',
         '여성연령대_50_일요일시간대_2_유동인구_수',
         '여성연령대_50_일요일시간대_3_유동인구_수',
         '여성연령대_50_일요일시간대_4_유동인구_수',
         '여성연령대_50_일요일시간대_5_유동인구_수',
         '여성연령대_50_일요일시간대_6_유동인구_수',
         '여성연령대_60_이상_월요일시간대_1_유동인구_수',
         '여성연령대_60_이상_월요일시간대_2_유동인구_수',
         '여성연령대_60_이상_월요일시간대_3_유동인구_수',
         '여성연령대_60_이상_월요일시간대_4_유동인구_수',
         '여성연령대_60_이상_월요일시간대_5_유동인구_수',
         '여성연령대_60_이상_월요일시간대_6_유동인구_수',
         '여성연령대_60_이상_화요일시간대_1_유동인구_수',
         '여성연령대_60_이상_화요일시간대_2_유동인구_수',
         '여성연령대_60_이상_화요일시간대_3_유동인구_수',
         '여성연령대_60_이상_화요일시간대_4_유동인구_수',
         '여성연령대_60_이상_화요일시간대_5_유동인구_수',
         '여성연령대_60_이상_화요일시간대_6_유동인구_수',
         '여성연령대_60_이상_수요일시간대_1_유동인구_수',
         '여성연령대_60_이상_수요일시간대_2_유동인구_수',
         '여성연령대_60_이상_수요일시간대_3_유동인구_수',
         '여성연령대_60_이상_수요일시간대_4_유동인구_수',
         '여성연령대_60_이상_수요일시간대_5_유동인구_수',
         '여성연령대_60_이상_수요일시간대_6_유동인구_수',
         '여성연령대_60_이상_목요일시간대_1_유동인구_수',
         '여성연령대_60_이상_목요일시간대_2_유동인구_수',
         '여성연령대_60_이상_목요일시간대_3_유동인구_수',
         '여성연령대_60_이상_목요일시간대_4_유동인구_수',
         '여성연령대_60_이상_목요일시간대_5_유동인구_수',
         '여성연령대_60_이상_목요일시간대_6_유동인구_수',
         '여성연령대_60_이상_금요일시간대_1_유동인구_수',
         '여성연령대_60_이상_금요일시간대_2_유동인구_수',
         '여성연령대_60_이상_금요일시간대_3_유동인구_수',
         '여성연령대_60_이상_금요일시간대_4_유동인구_수',
         '여성연령대_60_이상_금요일시간대_5_유동인구_수',
         '여성연령대_60_이상_금요일시간대_6_유동인구_수',
         '여성연령대_60_이상_토요일시간대_1_유동인구_수',
         '여성연령대_60_이상_토요일시간대_2_유동인구_수',
         '여성연령대_60_이상_토요일시간대_3_유동인구_수',
         '여성연령대_60_이상_토요일시간대_4_유동인구_수',
         '여성연령대_60_이상_토요일시간대_5_유동인구_수',
         '여성연령대_60_이상_토요일시간대_6_유동인구_수',
         '여성연령대_60_이상_일요일시간대_1_유동인구_수',
         '여성연령대_60_이상_일요일시간대_2_유동인구_수',
         '여성연령대_60_이상_일요일시간대_3_유동인구_수',
         '여성연령대_60_이상_일요일시간대_4_유동인구_수',
         '여성연령대_60_이상_일요일시간대_5_유동인구_수',
         '여성연령대_60_이상_일요일시간대_6_유동인구_수',
         '총_상주인구_수',
         '남성_상주인구_수',
         '여성_상주인구_수',
         '연령대_10_상주인구_수',
         '연령대_20_상주인구_수',
         '연령대_30_상주인구_수',
         '연령대_40_상주인구_수',
         '연령대_50_상주인구_수',
         '연령대_60_이상_상주인구_수',
         '남성연령대_10_상주인구_수',
         '남성연령대_20_상주인구_수',
         '남성연령대_30_상주인구_수',
         '남성연령대_40_상주인구_수',
         '남성연령대_50_상주인구_수',
         '남성연령대_60_이상_상주인구_수',
         '여성연령대_10_상주인구_수',
         '여성연령대_20_상주인구_수',
         '여성연령대_30_상주인구_수',
         '여성연령대_40_상주인구_수',
         '여성연령대_50_상주인구_수',
         '여성연령대_60_이상_상주인구_수',
         '총_가구_수',
         '아파트_가구_수',
         '비_아파트_가구_수',
         '아파트_단지_수',
         '아파트_면적_66_제곱미터_미만_세대_수',
         '아파트_면적_66_제곱미터_세대_수',
         '아파트_면적_99_제곱미터_세대_수',
         '아파트_면적_132_제곱미터_세대_수',
         '아파트_면적_165_제곱미터_세대_수',
         '아파트_가격_1_억_미만_세대_수',
         '아파트_가격_1_억_세대_수',
         '아파트_가격_2_억_세대_수',
         '아파트_가격_3_억_세대_수',
         '아파트_가격_4_억_세대_수',
         '아파트_가격_5_억_세대_수',
         '아파트_가격_6_억_이상_세대_수',
         '아파트_평균_면적',
         '아파트_평균_시가',
         '집객시설_수',
         '관공서_수',
         '은행_수',
         '종합병원_수',
         '일반_병원_수',
         '약국_수',
         '유치원_수',
         '초등학교_수',
         '중학교_수',
         '고등학교_수',
         '대학교_수',
         '백화점_수',
         '슈퍼마켓_수',
         '극장_수',
         '숙박_시설_수',
         '공항_수',
         '철도_역_수',
         '버스_터미널_수',
         '지하철_역_수',
         '버스_정거장_수'
]]

In [88]:
print(len(merge7_콜론정리.columns))
print(len(merge7.columns))

714
714


In [89]:
# 머신러닝 분석을 위해 콜론에 ' ~ ' 표시 다 ' - ' 로 변경
merge7_콜론정리.columns = [
    'adres',
         '도로명',
         '상권_코드_명',
         '서비스_업종_코드_명',
         '소분류업종',
         'storNm',
         'totalCount',
         'coordx',
         'coordy',
         'x',
         'y',
         'opbizYm',
         'clsbizYm',
         '영업일계산기준일',
         '상권_구분_코드',
         '상권_구분_코드_명',
         '상권_코드',
         '서비스_업종_코드',
         '분기매출금액',
         '당월_매출_건수',
         '엑스좌표_값',
         '와이좌표_값',
         '행정동_코드_명',
         '임대시세_1층',
         '임대시세_1층외',
         '소득분위', 
         '점포당분기매출',
         '임대시세_전체',
         '영업일수',
         '가구수',
         '소득분위점수',
         '점포수',
         '주중_매출_비율',
         '주말_매출_비율',
         '월요일_매출_비율',
         '화요일_매출_비율',
         '수요일_매출_비율',
         '목요일_매출_비율',
         '금요일_매출_비율',
         '토요일_매출_비율',
         '일요일_매출_비율',
         '시간대_00-06_매출_비율',
         '시간대_06-11_매출_비율',
         '시간대_11-14_매출_비율',
         '시간대_14-17_매출_비율',
         '시간대_17-21_매출_비율',
         '시간대_21-24_매출_비율',
         '남성_매출_비율',
         '여성_매출_비율',
         '연령대_10_매출_비율',
         '연령대_20_매출_비율',
         '연령대_30_매출_비율',
         '연령대_40_매출_비율',
         '연령대_50_매출_비율',
         '연령대_60_이상_매출_비율',
         '주중_매출_금액',
         '주말_매출_금액',
         '월요일_매출_금액',
         '화요일_매출_금액',
         '수요일_매출_금액',
         '목요일_매출_금액',
         '금요일_매출_금액',
         '토요일_매출_금액',
         '일요일_매출_금액',
         '시간대_00-06_매출_금액',
         '시간대_06-11_매출_금액',
         '시간대_11-14_매출_금액',
         '시간대_14-17_매출_금액',
         '시간대_17-21_매출_금액',
         '시간대_21-24_매출_금액',
         '남성_매출_금액',
         '여성_매출_금액',
         '연령대_10_매출_금액',
         '연령대_20_매출_금액',
         '연령대_30_매출_금액',
         '연령대_40_매출_금액',
         '연령대_50_매출_금액',
         '연령대_60_이상_매출_금액',
         '주중_매출_건수',
         '주말_매출_건수',
         '월요일_매출_건수',
         '화요일_매출_건수',
         '수요일_매출_건수',
         '목요일_매출_건수',
         '금요일_매출_건수',
         '토요일_매출_건수',
         '일요일_매출_건수',
         '시간대_건수-06_매출_건수',
         '시간대_건수-11_매출_건수',
         '시간대_건수-14_매출_건수',
         '시간대_건수-17_매출_건수',
         '시간대_건수-21_매출_건수',
         '시간대_건수-24_매출_건수',
         '남성_매출_건수',
         '여성_매출_건수',
         '연령대_10_매출_건수',
         '연령대_20_매출_건수',
         '연령대_30_매출_건수',
         '연령대_40_매출_건수',
         '연령대_50_매출_건수',
         '연령대_60_이상_매출_건수',
         '점포_수',
         '유사_업종_점포_수',
         '개업_율',
         '개업_점포_수',
         '폐업_률',
         '폐업_점포_수',
         '프랜차이즈_점포_수',
         '총_직장_인구_수',
         '남성_직장_인구_수',
         '여성_직장_인구_수',
         '연령대_10_직장_인구_수',
         '연령대_20_직장_인구_수',
         '연령대_30_직장_인구_수',
         '연령대_40_직장_인구_수',
         '연령대_50_직장_인구_수',
         '연령대_60_이상_직장_인구_수',
         '남성연령대_10_직장_인구_수',
         '남성연령대_20_직장_인구_수',
         '남성연령대_30_직장_인구_수',
         '남성연령대_40_직장_인구_수',
         '남성연령대_50_직장_인구_수',
         '남성연령대_60_이상_직장_인구_수',
         '여성연령대_10_직장_인구_수',
         '여성연령대_20_직장_인구_수',
         '여성연령대_30_직장_인구_수',
         '여성연령대_40_직장_인구_수',
         '여성연령대_50_직장_인구_수',
         '여성연령대_60_이상_직장_인구_수',
         '총_유동인구_수',
         '남성_유동인구_수',
         '여성_유동인구_수',
         '연령대_10_유동인구_수',
         '연령대_20_유동인구_수',
         '연령대_30_유동인구_수',
         '연령대_40_유동인구_수',
         '연령대_50_유동인구_수',
         '연령대_60_이상_유동인구_수',
         '시간대_1_유동인구_수',
         '시간대_2_유동인구_수',
         '시간대_3_유동인구_수',
         '시간대_4_유동인구_수',
         '시간대_5_유동인구_수',
         '시간대_6_유동인구_수',
         '월요일_유동인구_수',
         '화요일_유동인구_수',
         '수요일_유동인구_수',
         '목요일_유동인구_수',
         '금요일_유동인구_수',
         '토요일_유동인구_수',
         '일요일_유동인구_수',
         '남성연령대_10_월요일시간대_1_유동인구_수',
         '남성연령대_10_월요일시간대_2_유동인구_수',
         '남성연령대_10_월요일시간대_3_유동인구_수',
         '남성연령대_10_월요일시간대_4_유동인구_수',
         '남성연령대_10_월요일시간대_5_유동인구_수',
         '남성연령대_10_월요일시간대_6_유동인구_수',
         '남성연령대_10_화요일시간대_1_유동인구_수',
         '남성연령대_10_화요일시간대_2_유동인구_수',
         '남성연령대_10_화요일시간대_3_유동인구_수',
         '남성연령대_10_화요일시간대_4_유동인구_수',
         '남성연령대_10_화요일시간대_5_유동인구_수',
         '남성연령대_10_화요일시간대_6_유동인구_수',
         '남성연령대_10_수요일시간대_1_유동인구_수',
         '남성연령대_10_수요일시간대_2_유동인구_수',
         '남성연령대_10_수요일시간대_3_유동인구_수',
         '남성연령대_10_수요일시간대_4_유동인구_수',
         '남성연령대_10_수요일시간대_5_유동인구_수',
         '남성연령대_10_수요일시간대_6_유동인구_수',
         '남성연령대_10_목요일시간대_1_유동인구_수',
         '남성연령대_10_목요일시간대_2_유동인구_수',
         '남성연령대_10_목요일시간대_3_유동인구_수',
         '남성연령대_10_목요일시간대_4_유동인구_수',
         '남성연령대_10_목요일시간대_5_유동인구_수',
         '남성연령대_10_목요일시간대_6_유동인구_수',
         '남성연령대_10_금요일시간대_1_유동인구_수',
         '남성연령대_10_금요일시간대_2_유동인구_수',
         '남성연령대_10_금요일시간대_3_유동인구_수',
         '남성연령대_10_금요일시간대_4_유동인구_수',
         '남성연령대_10_금요일시간대_5_유동인구_수',
         '남성연령대_10_금요일시간대_6_유동인구_수',
         '남성연령대_10_토요일시간대_1_유동인구_수',
         '남성연령대_10_토요일시간대_2_유동인구_수',
         '남성연령대_10_토요일시간대_3_유동인구_수',
         '남성연령대_10_토요일시간대_4_유동인구_수',
         '남성연령대_10_토요일시간대_5_유동인구_수',
         '남성연령대_10_토요일시간대_6_유동인구_수',
         '남성연령대_10_일요일시간대_1_유동인구_수',
         '남성연령대_10_일요일시간대_2_유동인구_수',
         '남성연령대_10_일요일시간대_3_유동인구_수',
         '남성연령대_10_일요일시간대_4_유동인구_수',
         '남성연령대_10_일요일시간대_5_유동인구_수',
         '남성연령대_10_일요일시간대_6_유동인구_수',
         '남성연령대_20_월요일시간대_1_유동인구_수',
         '남성연령대_20_월요일시간대_2_유동인구_수',
         '남성연령대_20_월요일시간대_3_유동인구_수',
         '남성연령대_20_월요일시간대_4_유동인구_수',
         '남성연령대_20_월요일시간대_5_유동인구_수',
         '남성연령대_20_월요일시간대_6_유동인구_수',
         '남성연령대_20_화요일시간대_1_유동인구_수',
         '남성연령대_20_화요일시간대_2_유동인구_수',
         '남성연령대_20_화요일시간대_3_유동인구_수',
         '남성연령대_20_화요일시간대_4_유동인구_수',
         '남성연령대_20_화요일시간대_5_유동인구_수',
         '남성연령대_20_화요일시간대_6_유동인구_수',
         '남성연령대_20_수요일시간대_1_유동인구_수',
         '남성연령대_20_수요일시간대_2_유동인구_수',
         '남성연령대_20_수요일시간대_3_유동인구_수',
         '남성연령대_20_수요일시간대_4_유동인구_수',
         '남성연령대_20_수요일시간대_5_유동인구_수',
         '남성연령대_20_수요일시간대_6_유동인구_수',
         '남성연령대_20_목요일시간대_1_유동인구_수',
         '남성연령대_20_목요일시간대_2_유동인구_수',
         '남성연령대_20_목요일시간대_3_유동인구_수',
         '남성연령대_20_목요일시간대_4_유동인구_수',
         '남성연령대_20_목요일시간대_5_유동인구_수',
         '남성연령대_20_목요일시간대_6_유동인구_수',
         '남성연령대_20_금요일시간대_1_유동인구_수',
         '남성연령대_20_금요일시간대_2_유동인구_수',
         '남성연령대_20_금요일시간대_3_유동인구_수',
         '남성연령대_20_금요일시간대_4_유동인구_수',
         '남성연령대_20_금요일시간대_5_유동인구_수',
         '남성연령대_20_금요일시간대_6_유동인구_수',
         '남성연령대_20_토요일시간대_1_유동인구_수',
         '남성연령대_20_토요일시간대_2_유동인구_수',
         '남성연령대_20_토요일시간대_3_유동인구_수',
         '남성연령대_20_토요일시간대_4_유동인구_수',
         '남성연령대_20_토요일시간대_5_유동인구_수',
         '남성연령대_20_토요일시간대_6_유동인구_수',
         '남성연령대_20_일요일시간대_1_유동인구_수',
         '남성연령대_20_일요일시간대_2_유동인구_수',
         '남성연령대_20_일요일시간대_3_유동인구_수',
         '남성연령대_20_일요일시간대_4_유동인구_수',
         '남성연령대_20_일요일시간대_5_유동인구_수',
         '남성연령대_20_일요일시간대_6_유동인구_수',
         '남성연령대_30_월요일시간대_1_유동인구_수',
         '남성연령대_30_월요일시간대_2_유동인구_수',
         '남성연령대_30_월요일시간대_3_유동인구_수',
         '남성연령대_30_월요일시간대_4_유동인구_수',
         '남성연령대_30_월요일시간대_5_유동인구_수',
         '남성연령대_30_월요일시간대_6_유동인구_수',
         '남성연령대_30_화요일시간대_1_유동인구_수',
         '남성연령대_30_화요일시간대_2_유동인구_수',
         '남성연령대_30_화요일시간대_3_유동인구_수',
         '남성연령대_30_화요일시간대_4_유동인구_수',
         '남성연령대_30_화요일시간대_5_유동인구_수',
         '남성연령대_30_화요일시간대_6_유동인구_수',
         '남성연령대_30_수요일시간대_1_유동인구_수',
         '남성연령대_30_수요일시간대_2_유동인구_수',
         '남성연령대_30_수요일시간대_3_유동인구_수',
         '남성연령대_30_수요일시간대_4_유동인구_수',
         '남성연령대_30_수요일시간대_5_유동인구_수',
         '남성연령대_30_수요일시간대_6_유동인구_수',
         '남성연령대_30_목요일시간대_1_유동인구_수',
         '남성연령대_30_목요일시간대_2_유동인구_수',
         '남성연령대_30_목요일시간대_3_유동인구_수',
         '남성연령대_30_목요일시간대_4_유동인구_수',
         '남성연령대_30_목요일시간대_5_유동인구_수',
         '남성연령대_30_목요일시간대_6_유동인구_수',
         '남성연령대_30_금요일시간대_1_유동인구_수',
         '남성연령대_30_금요일시간대_2_유동인구_수',
         '남성연령대_30_금요일시간대_3_유동인구_수',
         '남성연령대_30_금요일시간대_4_유동인구_수',
         '남성연령대_30_금요일시간대_5_유동인구_수',
         '남성연령대_30_금요일시간대_6_유동인구_수',
         '남성연령대_30_토요일시간대_1_유동인구_수',
         '남성연령대_30_토요일시간대_2_유동인구_수',
         '남성연령대_30_토요일시간대_3_유동인구_수',
         '남성연령대_30_토요일시간대_4_유동인구_수',
         '남성연령대_30_토요일시간대_5_유동인구_수',
         '남성연령대_30_토요일시간대_6_유동인구_수',
         '남성연령대_30_일요일시간대_1_유동인구_수',
         '남성연령대_30_일요일시간대_2_유동인구_수',
         '남성연령대_30_일요일시간대_3_유동인구_수',
         '남성연령대_30_일요일시간대_4_유동인구_수',
         '남성연령대_30_일요일시간대_5_유동인구_수',
         '남성연령대_30_일요일시간대_6_유동인구_수',
         '남성연령대_40_월요일시간대_1_유동인구_수',
         '남성연령대_40_월요일시간대_2_유동인구_수',
         '남성연령대_40_월요일시간대_3_유동인구_수',
         '남성연령대_40_월요일시간대_4_유동인구_수',
         '남성연령대_40_월요일시간대_5_유동인구_수',
         '남성연령대_40_월요일시간대_6_유동인구_수',
         '남성연령대_40_화요일시간대_1_유동인구_수',
         '남성연령대_40_화요일시간대_2_유동인구_수',
         '남성연령대_40_화요일시간대_3_유동인구_수',
         '남성연령대_40_화요일시간대_4_유동인구_수',
         '남성연령대_40_화요일시간대_5_유동인구_수',
         '남성연령대_40_화요일시간대_6_유동인구_수',
         '남성연령대_40_수요일시간대_1_유동인구_수',
         '남성연령대_40_수요일시간대_2_유동인구_수',
         '남성연령대_40_수요일시간대_3_유동인구_수',
         '남성연령대_40_수요일시간대_4_유동인구_수',
         '남성연령대_40_수요일시간대_5_유동인구_수',
         '남성연령대_40_수요일시간대_6_유동인구_수',
         '남성연령대_40_목요일시간대_1_유동인구_수',
         '남성연령대_40_목요일시간대_2_유동인구_수',
         '남성연령대_40_목요일시간대_3_유동인구_수',
         '남성연령대_40_목요일시간대_4_유동인구_수',
         '남성연령대_40_목요일시간대_5_유동인구_수',
         '남성연령대_40_목요일시간대_6_유동인구_수',
         '남성연령대_40_금요일시간대_1_유동인구_수',
         '남성연령대_40_금요일시간대_2_유동인구_수',
         '남성연령대_40_금요일시간대_3_유동인구_수',
         '남성연령대_40_금요일시간대_4_유동인구_수',
         '남성연령대_40_금요일시간대_5_유동인구_수',
         '남성연령대_40_금요일시간대_6_유동인구_수',
         '남성연령대_40_토요일시간대_1_유동인구_수',
         '남성연령대_40_토요일시간대_2_유동인구_수',
         '남성연령대_40_토요일시간대_3_유동인구_수',
         '남성연령대_40_토요일시간대_4_유동인구_수',
         '남성연령대_40_토요일시간대_5_유동인구_수',
         '남성연령대_40_토요일시간대_6_유동인구_수',
         '남성연령대_40_일요일시간대_1_유동인구_수',
         '남성연령대_40_일요일시간대_2_유동인구_수',
         '남성연령대_40_일요일시간대_3_유동인구_수',
         '남성연령대_40_일요일시간대_4_유동인구_수',
         '남성연령대_40_일요일시간대_5_유동인구_수',
         '남성연령대_40_일요일시간대_6_유동인구_수',
         '남성연령대_50_월요일시간대_1_유동인구_수',
         '남성연령대_50_월요일시간대_2_유동인구_수',
         '남성연령대_50_월요일시간대_3_유동인구_수',
         '남성연령대_50_월요일시간대_4_유동인구_수',
         '남성연령대_50_월요일시간대_5_유동인구_수',
         '남성연령대_50_월요일시간대_6_유동인구_수',
         '남성연령대_50_화요일시간대_1_유동인구_수',
         '남성연령대_50_화요일시간대_2_유동인구_수',
         '남성연령대_50_화요일시간대_3_유동인구_수',
         '남성연령대_50_화요일시간대_4_유동인구_수',
         '남성연령대_50_화요일시간대_5_유동인구_수',
         '남성연령대_50_화요일시간대_6_유동인구_수',
         '남성연령대_50_수요일시간대_1_유동인구_수',
         '남성연령대_50_수요일시간대_2_유동인구_수',
         '남성연령대_50_수요일시간대_3_유동인구_수',
         '남성연령대_50_수요일시간대_4_유동인구_수',
         '남성연령대_50_수요일시간대_5_유동인구_수',
         '남성연령대_50_수요일시간대_6_유동인구_수',
         '남성연령대_50_목요일시간대_1_유동인구_수',
         '남성연령대_50_목요일시간대_2_유동인구_수',
         '남성연령대_50_목요일시간대_3_유동인구_수',
         '남성연령대_50_목요일시간대_4_유동인구_수',
         '남성연령대_50_목요일시간대_5_유동인구_수',
         '남성연령대_50_목요일시간대_6_유동인구_수',
         '남성연령대_50_금요일시간대_1_유동인구_수',
         '남성연령대_50_금요일시간대_2_유동인구_수',
         '남성연령대_50_금요일시간대_3_유동인구_수',
         '남성연령대_50_금요일시간대_4_유동인구_수',
         '남성연령대_50_금요일시간대_5_유동인구_수',
         '남성연령대_50_금요일시간대_6_유동인구_수',
         '남성연령대_50_토요일시간대_1_유동인구_수',
         '남성연령대_50_토요일시간대_2_유동인구_수',
         '남성연령대_50_토요일시간대_3_유동인구_수',
         '남성연령대_50_토요일시간대_4_유동인구_수',
         '남성연령대_50_토요일시간대_5_유동인구_수',
         '남성연령대_50_토요일시간대_6_유동인구_수',
         '남성연령대_50_일요일시간대_1_유동인구_수',
         '남성연령대_50_일요일시간대_2_유동인구_수',
         '남성연령대_50_일요일시간대_3_유동인구_수',
         '남성연령대_50_일요일시간대_4_유동인구_수',
         '남성연령대_50_일요일시간대_5_유동인구_수',
         '남성연령대_50_일요일시간대_6_유동인구_수',
         '남성연령대_60_이상_월요일시간대_1_유동인구_수',
         '남성연령대_60_이상_월요일시간대_2_유동인구_수',
         '남성연령대_60_이상_월요일시간대_3_유동인구_수',
         '남성연령대_60_이상_월요일시간대_4_유동인구_수',
         '남성연령대_60_이상_월요일시간대_5_유동인구_수',
         '남성연령대_60_이상_월요일시간대_6_유동인구_수',
         '남성연령대_60_이상_화요일시간대_1_유동인구_수',
         '남성연령대_60_이상_화요일시간대_2_유동인구_수',
         '남성연령대_60_이상_화요일시간대_3_유동인구_수',
         '남성연령대_60_이상_화요일시간대_4_유동인구_수',
         '남성연령대_60_이상_화요일시간대_5_유동인구_수',
         '남성연령대_60_이상_화요일시간대_6_유동인구_수',
         '남성연령대_60_이상_수요일시간대_1_유동인구_수',
         '남성연령대_60_이상_수요일시간대_2_유동인구_수',
         '남성연령대_60_이상_수요일시간대_3_유동인구_수',
         '남성연령대_60_이상_수요일시간대_4_유동인구_수',
         '남성연령대_60_이상_수요일시간대_5_유동인구_수',
         '남성연령대_60_이상_수요일시간대_6_유동인구_수',
         '남성연령대_60_이상_목요일시간대_1_유동인구_수',
         '남성연령대_60_이상_목요일시간대_2_유동인구_수',
         '남성연령대_60_이상_목요일시간대_3_유동인구_수',
         '남성연령대_60_이상_목요일시간대_4_유동인구_수',
         '남성연령대_60_이상_목요일시간대_5_유동인구_수',
         '남성연령대_60_이상_목요일시간대_6_유동인구_수',
         '남성연령대_60_이상_금요일시간대_1_유동인구_수',
         '남성연령대_60_이상_금요일시간대_2_유동인구_수',
         '남성연령대_60_이상_금요일시간대_3_유동인구_수',
         '남성연령대_60_이상_금요일시간대_4_유동인구_수',
         '남성연령대_60_이상_금요일시간대_5_유동인구_수',
         '남성연령대_60_이상_금요일시간대_6_유동인구_수',
         '남성연령대_60_이상_토요일시간대_1_유동인구_수',
         '남성연령대_60_이상_토요일시간대_2_유동인구_수',
         '남성연령대_60_이상_토요일시간대_3_유동인구_수',
         '남성연령대_60_이상_토요일시간대_4_유동인구_수',
         '남성연령대_60_이상_토요일시간대_5_유동인구_수',
         '남성연령대_60_이상_토요일시간대_6_유동인구_수',
         '남성연령대_60_이상_일요일시간대_1_유동인구_수',
         '남성연령대_60_이상_일요일시간대_2_유동인구_수',
         '남성연령대_60_이상_일요일시간대_3_유동인구_수',
         '남성연령대_60_이상_일요일시간대_4_유동인구_수',
         '남성연령대_60_이상_일요일시간대_5_유동인구_수',
         '남성연령대_60_이상_일요일시간대_6_유동인구_수',
         '여성연령대_10_월요일시간대_1_유동인구_수',
         '여성연령대_10_월요일시간대_2_유동인구_수',
         '여성연령대_10_월요일시간대_3_유동인구_수',
         '여성연령대_10_월요일시간대_4_유동인구_수',
         '여성연령대_10_월요일시간대_5_유동인구_수',
         '여성연령대_10_월요일시간대_6_유동인구_수',
         '여성연령대_10_화요일시간대_1_유동인구_수',
         '여성연령대_10_화요일시간대_2_유동인구_수',
         '여성연령대_10_화요일시간대_3_유동인구_수',
         '여성연령대_10_화요일시간대_4_유동인구_수',
         '여성연령대_10_화요일시간대_5_유동인구_수',
         '여성연령대_10_화요일시간대_6_유동인구_수',
         '여성연령대_10_수요일시간대_1_유동인구_수',
         '여성연령대_10_수요일시간대_2_유동인구_수',
         '여성연령대_10_수요일시간대_3_유동인구_수',
         '여성연령대_10_수요일시간대_4_유동인구_수',
         '여성연령대_10_수요일시간대_5_유동인구_수',
         '여성연령대_10_수요일시간대_6_유동인구_수',
         '여성연령대_10_목요일시간대_1_유동인구_수',
         '여성연령대_10_목요일시간대_2_유동인구_수',
         '여성연령대_10_목요일시간대_3_유동인구_수',
         '여성연령대_10_목요일시간대_4_유동인구_수',
         '여성연령대_10_목요일시간대_5_유동인구_수',
         '여성연령대_10_목요일시간대_6_유동인구_수',
         '여성연령대_10_금요일시간대_1_유동인구_수',
         '여성연령대_10_금요일시간대_2_유동인구_수',
         '여성연령대_10_금요일시간대_3_유동인구_수',
         '여성연령대_10_금요일시간대_4_유동인구_수',
         '여성연령대_10_금요일시간대_5_유동인구_수',
         '여성연령대_10_금요일시간대_6_유동인구_수',
         '여성연령대_10_토요일시간대_1_유동인구_수',
         '여성연령대_10_토요일시간대_2_유동인구_수',
         '여성연령대_10_토요일시간대_3_유동인구_수',
         '여성연령대_10_토요일시간대_4_유동인구_수',
         '여성연령대_10_토요일시간대_5_유동인구_수',
         '여성연령대_10_토요일시간대_6_유동인구_수',
         '여성연령대_10_일요일시간대_1_유동인구_수',
         '여성연령대_10_일요일시간대_2_유동인구_수',
         '여성연령대_10_일요일시간대_3_유동인구_수',
         '여성연령대_10_일요일시간대_4_유동인구_수',
         '여성연령대_10_일요일시간대_5_유동인구_수',
         '여성연령대_10_일요일시간대_6_유동인구_수',
         '여성연령대_20_월요일시간대_1_유동인구_수',
         '여성연령대_20_월요일시간대_2_유동인구_수',
         '여성연령대_20_월요일시간대_3_유동인구_수',
         '여성연령대_20_월요일시간대_4_유동인구_수',
         '여성연령대_20_월요일시간대_5_유동인구_수',
         '여성연령대_20_월요일시간대_6_유동인구_수',
         '여성연령대_20_화요일시간대_1_유동인구_수',
         '여성연령대_20_화요일시간대_2_유동인구_수',
         '여성연령대_20_화요일시간대_3_유동인구_수',
         '여성연령대_20_화요일시간대_4_유동인구_수',
         '여성연령대_20_화요일시간대_5_유동인구_수',
         '여성연령대_20_화요일시간대_6_유동인구_수',
         '여성연령대_20_수요일시간대_1_유동인구_수',
         '여성연령대_20_수요일시간대_2_유동인구_수',
         '여성연령대_20_수요일시간대_3_유동인구_수',
         '여성연령대_20_수요일시간대_4_유동인구_수',
         '여성연령대_20_수요일시간대_5_유동인구_수',
         '여성연령대_20_수요일시간대_6_유동인구_수',
         '여성연령대_20_목요일시간대_1_유동인구_수',
         '여성연령대_20_목요일시간대_2_유동인구_수',
         '여성연령대_20_목요일시간대_3_유동인구_수',
         '여성연령대_20_목요일시간대_4_유동인구_수',
         '여성연령대_20_목요일시간대_5_유동인구_수',
         '여성연령대_20_목요일시간대_6_유동인구_수',
         '여성연령대_20_금요일시간대_1_유동인구_수',
         '여성연령대_20_금요일시간대_2_유동인구_수',
         '여성연령대_20_금요일시간대_3_유동인구_수',
         '여성연령대_20_금요일시간대_4_유동인구_수',
         '여성연령대_20_금요일시간대_5_유동인구_수',
         '여성연령대_20_금요일시간대_6_유동인구_수',
         '여성연령대_20_토요일시간대_1_유동인구_수',
         '여성연령대_20_토요일시간대_2_유동인구_수',
         '여성연령대_20_토요일시간대_3_유동인구_수',
         '여성연령대_20_토요일시간대_4_유동인구_수',
         '여성연령대_20_토요일시간대_5_유동인구_수',
         '여성연령대_20_토요일시간대_6_유동인구_수',
         '여성연령대_20_일요일시간대_1_유동인구_수',
         '여성연령대_20_일요일시간대_2_유동인구_수',
         '여성연령대_20_일요일시간대_3_유동인구_수',
         '여성연령대_20_일요일시간대_4_유동인구_수',
         '여성연령대_20_일요일시간대_5_유동인구_수',
         '여성연령대_20_일요일시간대_6_유동인구_수',
         '여성연령대_30_월요일시간대_1_유동인구_수',
         '여성연령대_30_월요일시간대_2_유동인구_수',
         '여성연령대_30_월요일시간대_3_유동인구_수',
         '여성연령대_30_월요일시간대_4_유동인구_수',
         '여성연령대_30_월요일시간대_5_유동인구_수',
         '여성연령대_30_월요일시간대_6_유동인구_수',
         '여성연령대_30_화요일시간대_1_유동인구_수',
         '여성연령대_30_화요일시간대_2_유동인구_수',
         '여성연령대_30_화요일시간대_3_유동인구_수',
         '여성연령대_30_화요일시간대_4_유동인구_수',
         '여성연령대_30_화요일시간대_5_유동인구_수',
         '여성연령대_30_화요일시간대_6_유동인구_수',
         '여성연령대_30_수요일시간대_1_유동인구_수',
         '여성연령대_30_수요일시간대_2_유동인구_수',
         '여성연령대_30_수요일시간대_3_유동인구_수',
         '여성연령대_30_수요일시간대_4_유동인구_수',
         '여성연령대_30_수요일시간대_5_유동인구_수',
         '여성연령대_30_수요일시간대_6_유동인구_수',
         '여성연령대_30_목요일시간대_1_유동인구_수',
         '여성연령대_30_목요일시간대_2_유동인구_수',
         '여성연령대_30_목요일시간대_3_유동인구_수',
         '여성연령대_30_목요일시간대_4_유동인구_수',
         '여성연령대_30_목요일시간대_5_유동인구_수',
         '여성연령대_30_목요일시간대_6_유동인구_수',
         '여성연령대_30_금요일시간대_1_유동인구_수',
         '여성연령대_30_금요일시간대_2_유동인구_수',
         '여성연령대_30_금요일시간대_3_유동인구_수',
         '여성연령대_30_금요일시간대_4_유동인구_수',
         '여성연령대_30_금요일시간대_5_유동인구_수',
         '여성연령대_30_금요일시간대_6_유동인구_수',
         '여성연령대_30_토요일시간대_1_유동인구_수',
         '여성연령대_30_토요일시간대_2_유동인구_수',
         '여성연령대_30_토요일시간대_3_유동인구_수',
         '여성연령대_30_토요일시간대_4_유동인구_수',
         '여성연령대_30_토요일시간대_5_유동인구_수',
         '여성연령대_30_토요일시간대_6_유동인구_수',
         '여성연령대_30_일요일시간대_1_유동인구_수',
         '여성연령대_30_일요일시간대_2_유동인구_수',
         '여성연령대_30_일요일시간대_3_유동인구_수',
         '여성연령대_30_일요일시간대_4_유동인구_수',
         '여성연령대_30_일요일시간대_5_유동인구_수',
         '여성연령대_30_일요일시간대_6_유동인구_수',
         '여성연령대_40_월요일시간대_1_유동인구_수',
         '여성연령대_40_월요일시간대_2_유동인구_수',
         '여성연령대_40_월요일시간대_3_유동인구_수',
         '여성연령대_40_월요일시간대_4_유동인구_수',
         '여성연령대_40_월요일시간대_5_유동인구_수',
         '여성연령대_40_월요일시간대_6_유동인구_수',
         '여성연령대_40_화요일시간대_1_유동인구_수',
         '여성연령대_40_화요일시간대_2_유동인구_수',
         '여성연령대_40_화요일시간대_3_유동인구_수',
         '여성연령대_40_화요일시간대_4_유동인구_수',
         '여성연령대_40_화요일시간대_5_유동인구_수',
         '여성연령대_40_화요일시간대_6_유동인구_수',
         '여성연령대_40_수요일시간대_1_유동인구_수',
         '여성연령대_40_수요일시간대_2_유동인구_수',
         '여성연령대_40_수요일시간대_3_유동인구_수',
         '여성연령대_40_수요일시간대_4_유동인구_수',
         '여성연령대_40_수요일시간대_5_유동인구_수',
         '여성연령대_40_수요일시간대_6_유동인구_수',
         '여성연령대_40_목요일시간대_1_유동인구_수',
         '여성연령대_40_목요일시간대_2_유동인구_수',
         '여성연령대_40_목요일시간대_3_유동인구_수',
         '여성연령대_40_목요일시간대_4_유동인구_수',
         '여성연령대_40_목요일시간대_5_유동인구_수',
         '여성연령대_40_목요일시간대_6_유동인구_수',
         '여성연령대_40_금요일시간대_1_유동인구_수',
         '여성연령대_40_금요일시간대_2_유동인구_수',
         '여성연령대_40_금요일시간대_3_유동인구_수',
         '여성연령대_40_금요일시간대_4_유동인구_수',
         '여성연령대_40_금요일시간대_5_유동인구_수',
         '여성연령대_40_금요일시간대_6_유동인구_수',
         '여성연령대_40_토요일시간대_1_유동인구_수',
         '여성연령대_40_토요일시간대_2_유동인구_수',
         '여성연령대_40_토요일시간대_3_유동인구_수',
         '여성연령대_40_토요일시간대_4_유동인구_수',
         '여성연령대_40_토요일시간대_5_유동인구_수',
         '여성연령대_40_토요일시간대_6_유동인구_수',
         '여성연령대_40_일요일시간대_1_유동인구_수',
         '여성연령대_40_일요일시간대_2_유동인구_수',
         '여성연령대_40_일요일시간대_3_유동인구_수',
         '여성연령대_40_일요일시간대_4_유동인구_수',
         '여성연령대_40_일요일시간대_5_유동인구_수',
         '여성연령대_40_일요일시간대_6_유동인구_수',
         '여성연령대_50_월요일시간대_1_유동인구_수',
         '여성연령대_50_월요일시간대_2_유동인구_수',
         '여성연령대_50_월요일시간대_3_유동인구_수',
         '여성연령대_50_월요일시간대_4_유동인구_수',
         '여성연령대_50_월요일시간대_5_유동인구_수',
         '여성연령대_50_월요일시간대_6_유동인구_수',
         '여성연령대_50_화요일시간대_1_유동인구_수',
         '여성연령대_50_화요일시간대_2_유동인구_수',
         '여성연령대_50_화요일시간대_3_유동인구_수',
         '여성연령대_50_화요일시간대_4_유동인구_수',
         '여성연령대_50_화요일시간대_5_유동인구_수',
         '여성연령대_50_화요일시간대_6_유동인구_수',
         '여성연령대_50_수요일시간대_1_유동인구_수',
         '여성연령대_50_수요일시간대_2_유동인구_수',
         '여성연령대_50_수요일시간대_3_유동인구_수',
         '여성연령대_50_수요일시간대_4_유동인구_수',
         '여성연령대_50_수요일시간대_5_유동인구_수',
         '여성연령대_50_수요일시간대_6_유동인구_수',
         '여성연령대_50_목요일시간대_1_유동인구_수',
         '여성연령대_50_목요일시간대_2_유동인구_수',
         '여성연령대_50_목요일시간대_3_유동인구_수',
         '여성연령대_50_목요일시간대_4_유동인구_수',
         '여성연령대_50_목요일시간대_5_유동인구_수',
         '여성연령대_50_목요일시간대_6_유동인구_수',
         '여성연령대_50_금요일시간대_1_유동인구_수',
         '여성연령대_50_금요일시간대_2_유동인구_수',
         '여성연령대_50_금요일시간대_3_유동인구_수',
         '여성연령대_50_금요일시간대_4_유동인구_수',
         '여성연령대_50_금요일시간대_5_유동인구_수',
         '여성연령대_50_금요일시간대_6_유동인구_수',
         '여성연령대_50_토요일시간대_1_유동인구_수',
         '여성연령대_50_토요일시간대_2_유동인구_수',
         '여성연령대_50_토요일시간대_3_유동인구_수',
         '여성연령대_50_토요일시간대_4_유동인구_수',
         '여성연령대_50_토요일시간대_5_유동인구_수',
         '여성연령대_50_토요일시간대_6_유동인구_수',
         '여성연령대_50_일요일시간대_1_유동인구_수',
         '여성연령대_50_일요일시간대_2_유동인구_수',
         '여성연령대_50_일요일시간대_3_유동인구_수',
         '여성연령대_50_일요일시간대_4_유동인구_수',
         '여성연령대_50_일요일시간대_5_유동인구_수',
         '여성연령대_50_일요일시간대_6_유동인구_수',
         '여성연령대_60_이상_월요일시간대_1_유동인구_수',
         '여성연령대_60_이상_월요일시간대_2_유동인구_수',
         '여성연령대_60_이상_월요일시간대_3_유동인구_수',
         '여성연령대_60_이상_월요일시간대_4_유동인구_수',
         '여성연령대_60_이상_월요일시간대_5_유동인구_수',
         '여성연령대_60_이상_월요일시간대_6_유동인구_수',
         '여성연령대_60_이상_화요일시간대_1_유동인구_수',
         '여성연령대_60_이상_화요일시간대_2_유동인구_수',
         '여성연령대_60_이상_화요일시간대_3_유동인구_수',
         '여성연령대_60_이상_화요일시간대_4_유동인구_수',
         '여성연령대_60_이상_화요일시간대_5_유동인구_수',
         '여성연령대_60_이상_화요일시간대_6_유동인구_수',
         '여성연령대_60_이상_수요일시간대_1_유동인구_수',
         '여성연령대_60_이상_수요일시간대_2_유동인구_수',
         '여성연령대_60_이상_수요일시간대_3_유동인구_수',
         '여성연령대_60_이상_수요일시간대_4_유동인구_수',
         '여성연령대_60_이상_수요일시간대_5_유동인구_수',
         '여성연령대_60_이상_수요일시간대_6_유동인구_수',
         '여성연령대_60_이상_목요일시간대_1_유동인구_수',
         '여성연령대_60_이상_목요일시간대_2_유동인구_수',
         '여성연령대_60_이상_목요일시간대_3_유동인구_수',
         '여성연령대_60_이상_목요일시간대_4_유동인구_수',
         '여성연령대_60_이상_목요일시간대_5_유동인구_수',
         '여성연령대_60_이상_목요일시간대_6_유동인구_수',
         '여성연령대_60_이상_금요일시간대_1_유동인구_수',
         '여성연령대_60_이상_금요일시간대_2_유동인구_수',
         '여성연령대_60_이상_금요일시간대_3_유동인구_수',
         '여성연령대_60_이상_금요일시간대_4_유동인구_수',
         '여성연령대_60_이상_금요일시간대_5_유동인구_수',
         '여성연령대_60_이상_금요일시간대_6_유동인구_수',
         '여성연령대_60_이상_토요일시간대_1_유동인구_수',
         '여성연령대_60_이상_토요일시간대_2_유동인구_수',
         '여성연령대_60_이상_토요일시간대_3_유동인구_수',
         '여성연령대_60_이상_토요일시간대_4_유동인구_수',
         '여성연령대_60_이상_토요일시간대_5_유동인구_수',
         '여성연령대_60_이상_토요일시간대_6_유동인구_수',
         '여성연령대_60_이상_일요일시간대_1_유동인구_수',
         '여성연령대_60_이상_일요일시간대_2_유동인구_수',
         '여성연령대_60_이상_일요일시간대_3_유동인구_수',
         '여성연령대_60_이상_일요일시간대_4_유동인구_수',
         '여성연령대_60_이상_일요일시간대_5_유동인구_수',
         '여성연령대_60_이상_일요일시간대_6_유동인구_수',
         '총_상주인구_수',
         '남성_상주인구_수',
         '여성_상주인구_수',
         '연령대_10_상주인구_수',
         '연령대_20_상주인구_수',
         '연령대_30_상주인구_수',
         '연령대_40_상주인구_수',
         '연령대_50_상주인구_수',
         '연령대_60_이상_상주인구_수',
         '남성연령대_10_상주인구_수',
         '남성연령대_20_상주인구_수',
         '남성연령대_30_상주인구_수',
         '남성연령대_40_상주인구_수',
         '남성연령대_50_상주인구_수',
         '남성연령대_60_이상_상주인구_수',
         '여성연령대_10_상주인구_수',
         '여성연령대_20_상주인구_수',
         '여성연령대_30_상주인구_수',
         '여성연령대_40_상주인구_수',
         '여성연령대_50_상주인구_수',
         '여성연령대_60_이상_상주인구_수',
         '총_가구_수',
         '아파트_가구_수',
         '비_아파트_가구_수',
         '아파트_단지_수',
         '아파트_면적_66_제곱미터_미만_세대_수',
         '아파트_면적_66_제곱미터_세대_수',
         '아파트_면적_99_제곱미터_세대_수',
         '아파트_면적_132_제곱미터_세대_수',
         '아파트_면적_165_제곱미터_세대_수',
         '아파트_가격_1_억_미만_세대_수',
         '아파트_가격_1_억_세대_수',
         '아파트_가격_2_억_세대_수',
         '아파트_가격_3_억_세대_수',
         '아파트_가격_4_억_세대_수',
         '아파트_가격_5_억_세대_수',
         '아파트_가격_6_억_이상_세대_수',
         '아파트_평균_면적',
         '아파트_평균_시가',
         '집객시설_수',
         '관공서_수',
         '은행_수',
         '종합병원_수',
         '일반_병원_수',
         '약국_수',
         '유치원_수',
         '초등학교_수',
         '중학교_수',
         '고등학교_수',
         '대학교_수',
         '백화점_수',
         '슈퍼마켓_수',
         '극장_수',
         '숙박_시설_수',
         '공항_수',
         '철도_역_수',
         '버스_터미널_수',
         '지하철_역_수',
         '버스_정거장_수'
]

In [90]:
print(len(merge7_콜론정리.columns))
print(len(merge7.columns))

714
714


In [92]:
# 가점 target 만들기
merge7_콜론정리.head(1)

,adres,도로명,상권_코드_명,서비스_업종_코드_명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,상권_구분_코드,상권_구분_코드_명,상권_코드,서비스_업종_코드,분기매출금액,당월_매출_건수,엑스좌표_값,와이좌표_값,행정동_코드_명,임대시세_1층,임대시세_1층외,소득분위,점포당분기매출,임대시세_전체,영업일수,가구수,소득분위점수,점포수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00-06_매출_비율,시간대_06-11_매출_비율,시간대_11-14_매출_비율,시간대_14-17_매출_비율,시간대_17-21_매출_비율,시간대_21-24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00-06_매출_금액,시간대_06-11_매출_금액,시간대_11-14_매출_금액,시간대_14-17_매출_금액,시간대_17-21_매출_금액,시간대_21-24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수-06_매출_건수,시간대_건수-11_매출_건수,시간대_건수-14_매출_건수,시간대_건수-17_매출_건수,시간대_건수-21_매출_건수,시간대_건수-24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,시간대_1_유동인구_수,시간대_2_유동인구_수,시간대_3_유동인구_수,시간대_4_유동인구_수,시간대_5_유동인구_수,시간대_6_유동인구_수,월요일_유동인구_수,화요일_유동인구_수,수요일_유동인구_수,목요일_유동인구_수,금요일_유동인구_수,토요일_유동인구_수,일요일_유동인구_수,남성연령대_10_월요일시간대_1_유동인구_수,남성연령대_10_월요일시간대_2_유동인구_수,남성연령대_10_월요일시간대_3_유동인구_수,남성연령대_10_월요일시간대_4_유동인구_수,남성연령대_10_월요일시간대_5_유동인구_수,남성연령대_10_월요일시간대_6_유동인구_수,남성연령대_10_화요일시간대_1_유동인구_수,남성연령대_10_화요일시간대_2_유동인구_수,남성연령대_10_화요일시간대_3_유동인구_수,남성연령대_10_화요일시간대_4_유동인구_수,남성연령대_10_화요일시간대_5_유동인구_수,남성연령대_10_화요일시간대_6_유동인구_수,남성연령대_10_수요일시간대_1_유동인구_수,남성연령대_10_수요일시간대_2_유동인구_수,남성연령대_10_수요일시간대_3_유동인구_수,남성연령대_10_수요일시간대_4_유동인구_수,남성연령대_10_수요일시간대_5_유동인구_수,남성연령대_10_수요일시간대_6_유동인구_수,남성연령대_10_목요일시간대_1_유동인구_수,남성연령대_10_목요일시간대_2_유동인구_수,남성연령대_10_목요일시간대_3_유동인구_수,남성연령대_10_목요일시간대_4_유동인구_수,남성연령대_10_목요일시간대_5_유동인구_수,남성연령대_10_목요일시간대_6_유동인구_수,남성연령대_10_금요일시간대_1_유동인구_수,남성연령대_10_금요일시간대_2_유동인구_수,남성연령대_10_금요일시간대_3_유동인구_수,남성연령대_10_금요일시간대_4_유동인구_수,남성연령대_10_금요일시간대_5_유동인구_수,남성연령대_10_금요일시간대_6_유동인구_수,남성연령대_10_토요일시간대_1_유동인구_수,남성연령대_10_토요일시간대_2_유동인구_수,남성연령대_10_토요일시간대_3_유동인구_수,남성연령대_10_토요일시간대_4_유동인구_수,남성연령대_10_토요일시간대_5_유동인구_수,남성연령대_10_토요일시간대_6_유동인구_수,남성연령대_10_일요일시간대_1_유동인구_수,남성연령대_10_일요일시간대_2_유동인구_수,남성연령대_10_일요일시간대_3_유동인구_수,남성연령대_10_일요일시간대_4_유동인구_수,남성연령대_10_일요일시간대_5_유동인구_수,남성연령대_10_일요일시간대_6_유동인구_수,남성연령대_20_월요일시간대_1_유동인구_수,남성연령대_20_월요일시간대_2_유동인구_수,남성연령대_20_월요일시간대_3_유동인구_수,남성연령대_20_월요일시간대_4_유동인구_수,남성연령대_20_월요일시간대_5_유동인구_수,남성연령대_20_월요일시간대_6_유동인구_수,남성연령대_20_화요일시간대_1_유동인구_수,남성연령대_20_화요일시간대_2_유동인구_수,남성연령대_20_화요일시간대_3_유동인구_수,남성연령대_20_화요일시간대_4_유동인구_수,남성연령대_20_화요일시간대_5_유동인구_수,남성연령대_20_화요일시간대_6_유동인구_수,남성연령대_20_수요일시간대_1_유동인구_수,남성연령대_20_수요일시간대_2_유동인구_수,남성연령대_20_수요일시간대_3_유동인구_수,남성연령대_20_수요일시간대_4_유동인구_수,남성연령대_20_수요일시간대_5_유동인구_수,남성연령대_20_수요일시간대_6_유동인구_수,남성연령대_20_목요일시간대_1_유동인구_수,남성연령대_20_목요일시간대_2_유동인구_수,남성연령대_20_목요일시간대_3_유동인구_수,남성연령대_20_목요일시간대_4_유동인구_수,남성연령대_20_목요일시간대_5_유동인구_수,남성연령대_20_목요일시간대_6_유동인구_수,남성연령대_20_금요일시간대_1_유동인구_수,남성연령대_20_금요일시간대_2_유동인구_수,남성연령대_20_금요일시간대_3_유동인구_수,남성연령대_20_금요일시간대_4_유동인구_수,남성연령대_20_금요일시간대_5_유동인구_수,남성연령대_20_금요일시간대_6_유동인구_수,남성연령대_20_토요일시간대_1_유동인구_수,남성연령대_20_토요일시간대_2_유동인구_수,남성연령대_20_토요일시간대_3_유동인구_수,남성연령대_20_토요일시간대_4_유동인구_수,남성연령대_20_토요일시간대_5_유동인구_수,남성연령대_20_토요일시간대_6_유동인구_수,남성연령대_20_일요일시간대_1_유동인구_수,남성연령대_20_일요일시간대_2_유동인구_수,남성연령대_20_일요일시간대_3_유동인구_수,남성연령대_20_일요일시간대_4_유동인구_수,남성연령대_20_일요일시간대_5_유동인구_수,남성연령대_20_일요일시간대_6_유동인구_수,남성연령대_30_월요일시간대_1_유동인구_수,남성연령대_30_월요일시간대_2_유동인구_수,남성연령대_30_월요일시간대_3_유동인구_수,남성연령대_30_월요일시간대_4_유동인구_수,남성연령대_30_월요일시간대_5_유동인구_수,남성연령대_30_월요일시간대_6_유동인구_수,남성연령대_30_화

In [91]:
merge7_콜론정리.to_pickle('merge_이력_매출_최종.pki')
pd.read_pickle('merge_이력_매출_최종.pki')

,adres,도로명,상권_코드_명,서비스_업종_코드_명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,상권_구분_코드,상권_구분_코드_명,상권_코드,서비스_업종_코드,분기매출금액,당월_매출_건수,엑스좌표_값,와이좌표_값,행정동_코드_명,임대시세_1층,임대시세_1층외,소득분위,점포당분기매출,임대시세_전체,영업일수,가구수,소득분위점수,점포수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00-06_매출_비율,시간대_06-11_매출_비율,시간대_11-14_매출_비율,시간대_14-17_매출_비율,시간대_17-21_매출_비율,시간대_21-24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00-06_매출_금액,시간대_06-11_매출_금액,시간대_11-14_매출_금액,시간대_14-17_매출_금액,시간대_17-21_매출_금액,시간대_21-24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수-06_매출_건수,시간대_건수-11_매출_건수,시간대_건수-14_매출_건수,시간대_건수-17_매출_건수,시간대_건수-21_매출_건수,시간대_건수-24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,시간대_1_유동인구_수,시간대_2_유동인구_수,시간대_3_유동인구_수,시간대_4_유동인구_수,시간대_5_유동인구_수,시간대_6_유동인구_수,월요일_유동인구_수,화요일_유동인구_수,수요일_유동인구_수,목요일_유동인구_수,금요일_유동인구_수,토요일_유동인구_수,일요일_유동인구_수,남성연령대_10_월요일시간대_1_유동인구_수,남성연령대_10_월요일시간대_2_유동인구_수,남성연령대_10_월요일시간대_3_유동인구_수,남성연령대_10_월요일시간대_4_유동인구_수,남성연령대_10_월요일시간대_5_유동인구_수,남성연령대_10_월요일시간대_6_유동인구_수,남성연령대_10_화요일시간대_1_유동인구_수,남성연령대_10_화요일시간대_2_유동인구_수,남성연령대_10_화요일시간대_3_유동인구_수,남성연령대_10_화요일시간대_4_유동인구_수,남성연령대_10_화요일시간대_5_유동인구_수,남성연령대_10_화요일시간대_6_유동인구_수,남성연령대_10_수요일시간대_1_유동인구_수,남성연령대_10_수요일시간대_2_유동인구_수,남성연령대_10_수요일시간대_3_유동인구_수,남성연령대_10_수요일시간대_4_유동인구_수,남성연령대_10_수요일시간대_5_유동인구_수,남성연령대_10_수요일시간대_6_유동인구_수,남성연령대_10_목요일시간대_1_유동인구_수,남성연령대_10_목요일시간대_2_유동인구_수,남성연령대_10_목요일시간대_3_유동인구_수,남성연령대_10_목요일시간대_4_유동인구_수,남성연령대_10_목요일시간대_5_유동인구_수,남성연령대_10_목요일시간대_6_유동인구_수,남성연령대_10_금요일시간대_1_유동인구_수,남성연령대_10_금요일시간대_2_유동인구_수,남성연령대_10_금요일시간대_3_유동인구_수,남성연령대_10_금요일시간대_4_유동인구_수,남성연령대_10_금요일시간대_5_유동인구_수,남성연령대_10_금요일시간대_6_유동인구_수,남성연령대_10_토요일시간대_1_유동인구_수,남성연령대_10_토요일시간대_2_유동인구_수,남성연령대_10_토요일시간대_3_유동인구_수,남성연령대_10_토요일시간대_4_유동인구_수,남성연령대_10_토요일시간대_5_유동인구_수,남성연령대_10_토요일시간대_6_유동인구_수,남성연령대_10_일요일시간대_1_유동인구_수,남성연령대_10_일요일시간대_2_유동인구_수,남성연령대_10_일요일시간대_3_유동인구_수,남성연령대_10_일요일시간대_4_유동인구_수,남성연령대_10_일요일시간대_5_유동인구_수,남성연령대_10_일요일시간대_6_유동인구_수,남성연령대_20_월요일시간대_1_유동인구_수,남성연령대_20_월요일시간대_2_유동인구_수,남성연령대_20_월요일시간대_3_유동인구_수,남성연령대_20_월요일시간대_4_유동인구_수,남성연령대_20_월요일시간대_5_유동인구_수,남성연령대_20_월요일시간대_6_유동인구_수,남성연령대_20_화요일시간대_1_유동인구_수,남성연령대_20_화요일시간대_2_유동인구_수,남성연령대_20_화요일시간대_3_유동인구_수,남성연령대_20_화요일시간대_4_유동인구_수,남성연령대_20_화요일시간대_5_유동인구_수,남성연령대_20_화요일시간대_6_유동인구_수,남성연령대_20_수요일시간대_1_유동인구_수,남성연령대_20_수요일시간대_2_유동인구_수,남성연령대_20_수요일시간대_3_유동인구_수,남성연령대_20_수요일시간대_4_유동인구_수,남성연령대_20_수요일시간대_5_유동인구_수,남성연령대_20_수요일시간대_6_유동인구_수,남성연령대_20_목요일시간대_1_유동인구_수,남성연령대_20_목요일시간대_2_유동인구_수,남성연령대_20_목요일시간대_3_유동인구_수,남성연령대_20_목요일시간대_4_유동인구_수,남성연령대_20_목요일시간대_5_유동인구_수,남성연령대_20_목요일시간대_6_유동인구_수,남성연령대_20_금요일시간대_1_유동인구_수,남성연령대_20_금요일시간대_2_유동인구_수,남성연령대_20_금요일시간대_3_유동인구_수,남성연령대_20_금요일시간대_4_유동인구_수,남성연령대_20_금요일시간대_5_유동인구_수,남성연령대_20_금요일시간대_6_유동인구_수,남성연령대_20_토요일시간대_1_유동인구_수,남성연령대_20_토요일시간대_2_유동인구_수,남성연령대_20_토요일시간대_3_유동인구_수,남성연령대_20_토요일시간대_4_유동인구_수,남성연령대_20_토요일시간대_5_유동인구_수,남성연령대_20_토요일시간대_6_유동인구_수,남성연령대_20_일요일시간대_1_유동인구_수,남성연령대_20_일요일시간대_2_유동인구_수,남성연령대_20_일요일시간대_3_유동인구_수,남성연령대_20_일요일시간대_4_유동인구_수,남성연령대_20_일요일시간대_5_유동인구_수,남성연령대_20_일요일시간대_6_유동인구_수,남성연령대_30_월요일시간대_1_유동인구_수,남성연령대_30_월요일시간대_2_유동인구_수,남성연령대_30_월요일시간대_3_유동인구_수,남성연령대_30_월요일시간대_4_유동인구_수,남성연령대_30_월요일시간대_5_유동인구_수,남성연령대_30_월요일시간대_6_유동인구_수,남성연령대_30_화

In [93]:
ML_model= pd.read_pickle('merge_이력_매출_최종.pki')

In [100]:
len(ML_model.columns)

714

In [95]:
col_lis = []
for i in ML_model.columns:
    col_lis.append(i)
col_lis

['adres',
 '도로명',
 '상권_코드_명',
 '서비스_업종_코드_명',
 '소분류업종',
 'storNm',
 'totalCount',
 'coordx',
 'coordy',
 'x',
 'y',
 'opbizYm',
 'clsbizYm',
 '영업일계산기준일',
 '상권_구분_코드',
 '상권_구분_코드_명',
 '상권_코드',
 '서비스_업종_코드',
 '분기매출금액',
 '당월_매출_건수',
 '엑스좌표_값',
 '와이좌표_값',
 '행정동_코드_명',
 '임대시세_1층',
 '임대시세_1층외',
 '소득분위',
 '점포당분기매출',
 '임대시세_전체',
 '영업일수',
 '가구수',
 '소득분위점수',
 '점포수',
 '주중_매출_비율',
 '주말_매출_비율',
 '월요일_매출_비율',
 '화요일_매출_비율',
 '수요일_매출_비율',
 '목요일_매출_비율',
 '금요일_매출_비율',
 '토요일_매출_비율',
 '일요일_매출_비율',
 '시간대_00-06_매출_비율',
 '시간대_06-11_매출_비율',
 '시간대_11-14_매출_비율',
 '시간대_14-17_매출_비율',
 '시간대_17-21_매출_비율',
 '시간대_21-24_매출_비율',
 '남성_매출_비율',
 '여성_매출_비율',
 '연령대_10_매출_비율',
 '연령대_20_매출_비율',
 '연령대_30_매출_비율',
 '연령대_40_매출_비율',
 '연령대_50_매출_비율',
 '연령대_60_이상_매출_비율',
 '주중_매출_금액',
 '주말_매출_금액',
 '월요일_매출_금액',
 '화요일_매출_금액',
 '수요일_매출_금액',
 '목요일_매출_금액',
 '금요일_매출_금액',
 '토요일_매출_금액',
 '일요일_매출_금액',
 '시간대_00-06_매출_금액',
 '시간대_06-11_매출_금액',
 '시간대_11-14_매출_금액',
 '시간대_14-17_매출_금액',
 '시간대_17-21_매출_금액',
 '시간대_21-24_매출_금액',
 '남성_매출_금액',
 '여성_매출_금

In [97]:
new_col_lis = []
for i in col_lis:
    c = i.replace('_','')
    new_col_lis.append(c)
new_col_lis    

['adres',
 '도로명',
 '상권코드명',
 '서비스업종코드명',
 '소분류업종',
 'storNm',
 'totalCount',
 'coordx',
 'coordy',
 'x',
 'y',
 'opbizYm',
 'clsbizYm',
 '영업일계산기준일',
 '상권구분코드',
 '상권구분코드명',
 '상권코드',
 '서비스업종코드',
 '분기매출금액',
 '당월매출건수',
 '엑스좌표값',
 '와이좌표값',
 '행정동코드명',
 '임대시세1층',
 '임대시세1층외',
 '소득분위',
 '점포당분기매출',
 '임대시세전체',
 '영업일수',
 '가구수',
 '소득분위점수',
 '점포수',
 '주중매출비율',
 '주말매출비율',
 '월요일매출비율',
 '화요일매출비율',
 '수요일매출비율',
 '목요일매출비율',
 '금요일매출비율',
 '토요일매출비율',
 '일요일매출비율',
 '시간대00-06매출비율',
 '시간대06-11매출비율',
 '시간대11-14매출비율',
 '시간대14-17매출비율',
 '시간대17-21매출비율',
 '시간대21-24매출비율',
 '남성매출비율',
 '여성매출비율',
 '연령대10매출비율',
 '연령대20매출비율',
 '연령대30매출비율',
 '연령대40매출비율',
 '연령대50매출비율',
 '연령대60이상매출비율',
 '주중매출금액',
 '주말매출금액',
 '월요일매출금액',
 '화요일매출금액',
 '수요일매출금액',
 '목요일매출금액',
 '금요일매출금액',
 '토요일매출금액',
 '일요일매출금액',
 '시간대00-06매출금액',
 '시간대06-11매출금액',
 '시간대11-14매출금액',
 '시간대14-17매출금액',
 '시간대17-21매출금액',
 '시간대21-24매출금액',
 '남성매출금액',
 '여성매출금액',
 '연령대10매출금액',
 '연령대20매출금액',
 '연령대30매출금액',
 '연령대40매출금액',
 '연령대50매출금액',
 '연령대60이상매출금액',
 '주중매출건수',
 '주말매출건수',
 '월요일매출건수

In [98]:
col_lis_fi = []
for i in new_col_lis:
    c = i.replace('-','')
    col_lis_fi.append(c)
col_lis_fi 

['adres',
 '도로명',
 '상권코드명',
 '서비스업종코드명',
 '소분류업종',
 'storNm',
 'totalCount',
 'coordx',
 'coordy',
 'x',
 'y',
 'opbizYm',
 'clsbizYm',
 '영업일계산기준일',
 '상권구분코드',
 '상권구분코드명',
 '상권코드',
 '서비스업종코드',
 '분기매출금액',
 '당월매출건수',
 '엑스좌표값',
 '와이좌표값',
 '행정동코드명',
 '임대시세1층',
 '임대시세1층외',
 '소득분위',
 '점포당분기매출',
 '임대시세전체',
 '영업일수',
 '가구수',
 '소득분위점수',
 '점포수',
 '주중매출비율',
 '주말매출비율',
 '월요일매출비율',
 '화요일매출비율',
 '수요일매출비율',
 '목요일매출비율',
 '금요일매출비율',
 '토요일매출비율',
 '일요일매출비율',
 '시간대0006매출비율',
 '시간대0611매출비율',
 '시간대1114매출비율',
 '시간대1417매출비율',
 '시간대1721매출비율',
 '시간대2124매출비율',
 '남성매출비율',
 '여성매출비율',
 '연령대10매출비율',
 '연령대20매출비율',
 '연령대30매출비율',
 '연령대40매출비율',
 '연령대50매출비율',
 '연령대60이상매출비율',
 '주중매출금액',
 '주말매출금액',
 '월요일매출금액',
 '화요일매출금액',
 '수요일매출금액',
 '목요일매출금액',
 '금요일매출금액',
 '토요일매출금액',
 '일요일매출금액',
 '시간대0006매출금액',
 '시간대0611매출금액',
 '시간대1114매출금액',
 '시간대1417매출금액',
 '시간대1721매출금액',
 '시간대2124매출금액',
 '남성매출금액',
 '여성매출금액',
 '연령대10매출금액',
 '연령대20매출금액',
 '연령대30매출금액',
 '연령대40매출금액',
 '연령대50매출금액',
 '연령대60이상매출금액',
 '주중매출건수',
 '주말매출건수',
 '월요일매출건수',
 '화요일매출건수

In [99]:
len(col_lis_fi)

714

In [101]:
# 머신러닝을 위해 col dash 다 지우기
ML_model.columns = col_lis_fi
ML_model

,adres,도로명,상권코드명,서비스업종코드명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,상권구분코드,상권구분코드명,상권코드,서비스업종코드,분기매출금액,당월매출건수,엑스좌표값,와이좌표값,행정동코드명,임대시세1층,임대시세1층외,소득분위,점포당분기매출,임대시세전체,영업일수,가구수,소득분위점수,점포수,주중매출비율,주말매출비율,월요일매출비율,화요일매출비율,수요일매출비율,목요일매출비율,금요일매출비율,토요일매출비율,일요일매출비율,시간대0006매출비율,시간대0611매출비율,시간대1114매출비율,시간대1417매출비율,시간대1721매출비율,시간대2124매출비율,남성매출비율,여성매출비율,연령대10매출비율,연령대20매출비율,연령대30매출비율,연령대40매출비율,연령대50매출비율,연령대60이상매출비율,주중매출금액,주말매출금액,월요일매출금액,화요일매출금액,수요일매출금액,목요일매출금액,금요일매출금액,토요일매출금액,일요일매출금액,시간대0006매출금액,시간대0611매출금액,시간대1114매출금액,시간대1417매출금액,시간대1721매출금액,시간대2124매출금액,남성매출금액,여성매출금액,연령대10매출금액,연령대20매출금액,연령대30매출금액,연령대40매출금액,연령대50매출금액,연령대60이상매출금액,주중매출건수,주말매출건수,월요일매출건수,화요일매출건수,수요일매출건수,목요일매출건수,금요일매출건수,토요일매출건수,일요일매출건수,시간대건수06매출건수,시간대건수11매출건수,시간대건수14매출건수,시간대건수17매출건수,시간대건수21매출건수,시간대건수24매출건수,남성매출건수,여성매출건수,연령대10매출건수,연령대20매출건수,연령대30매출건수,연령대40매출건수,연령대50매출건수,연령대60이상매출건수,점포수,유사업종점포수,개업율,개업점포수,폐업률,폐업점포수,프랜차이즈점포수,총직장인구수,남성직장인구수,여성직장인구수,연령대10직장인구수,연령대20직장인구수,연령대30직장인구수,연령대40직장인구수,연령대50직장인구수,연령대60이상직장인구수,남성연령대10직장인구수,남성연령대20직장인구수,남성연령대30직장인구수,남성연령대40직장인구수,남성연령대50직장인구수,남성연령대60이상직장인구수,여성연령대10직장인구수,여성연령대20직장인구수,여성연령대30직장인구수,여성연령대40직장인구수,여성연령대50직장인구수,여성연령대60이상직장인구수,총유동인구수,남성유동인구수,여성유동인구수,연령대10유동인구수,연령대20유동인구수,연령대30유동인구수,연령대40유동인구수,연령대50유동인구수,연령대60이상유동인구수,시간대1유동인구수,시간대2유동인구수,시간대3유동인구수,시간대4유동인구수,시간대5유동인구수,시간대6유동인구수,월요일유동인구수,화요일유동인구수,수요일유동인구수,목요일유동인구수,금요일유동인구수,토요일유동인구수,일요일유동인구수,남성연령대10월요일시간대1유동인구수,남성연령대10월요일시간대2유동인구수,남성연령대10월요일시간대3유동인구수,남성연령대10월요일시간대4유동인구수,남성연령대10월요일시간대5유동인구수,남성연령대10월요일시간대6유동인구수,남성연령대10화요일시간대1유동인구수,남성연령대10화요일시간대2유동인구수,남성연령대10화요일시간대3유동인구수,남성연령대10화요일시간대4유동인구수,남성연령대10화요일시간대5유동인구수,남성연령대10화요일시간대6유동인구수,남성연령대10수요일시간대1유동인구수,남성연령대10수요일시간대2유동인구수,남성연령대10수요일시간대3유동인구수,남성연령대10수요일시간대4유동인구수,남성연령대10수요일시간대5유동인구수,남성연령대10수요일시간대6유동인구수,남성연령대10목요일시간대1유동인구수,남성연령대10목요일시간대2유동인구수,남성연령대10목요일시간대3유동인구수,남성연령대10목요일시간대4유동인구수,남성연령대10목요일시간대5유동인구수,남성연령대10목요일시간대6유동인구수,남성연령대10금요일시간대1유동인구수,남성연령대10금요일시간대2유동인구수,남성연령대10금요일시간대3유동인구수,남성연령대10금요일시간대4유동인구수,남성연령대10금요일시간대5유동인구수,남성연령대10금요일시간대6유동인구수,남성연령대10토요일시간대1유동인구수,남성연령대10토요일시간대2유동인구수,남성연령대10토요일시간대3유동인구수,남성연령대10토요일시간대4유동인구수,남성연령대10토요일시간대5유동인구수,남성연령대10토요일시간대6유동인구수,남성연령대10일요일시간대1유동인구수,남성연령대10일요일시간대2유동인구수,남성연령대10일요일시간대3유동인구수,남성연령대10일요일시간대4유동인구수,남성연령대10일요일시간대5유동인구수,남성연령대10일요일시간대6유동인구수,남성연령대20월요일시간대1유동인구수,남성연령대20월요일시간대2유동인구수,남성연령대20월요일시간대3유동인구수,남성연령대20월요일시간대4유동인구수,남성연령대20월요일시간대5유동인구수,남성연령대20월요일시간대6유동인구수,남성연령대20화요일시간대1유동인구수,남성연령대20화요일시간대2유동인구수,남성연령대20화요일시간대3유동인구수,남성연령대20화요일시간대4유동인구수,남성연령대20화요일시간대5유동인구수,남성연령대20화요일시간대6유동인구수,남성연령대20수요일시간대1유동인구수,남성연령대20수요일시간대2유동인구수,남성연령대20수요일시간대3유동인구수,남성연령대20수요일시간대4유동인구수,남성연령대20수요일시간대5유동인구수,남성연령대20수요일시간대6유동인구수,남성연령대20목요일시간대1유동인구수,남성연령대20목요일시간대2유동인구수,남성연령대20목요일시간대3유동인구수,남성연령대20목요일시간대4유동인구수,남성연령대20목요일시간대5유동인구수,남성연령대20목요일시간대6유동인구수,남성연령대20금요일시간대1유동인구수,남성연령대20금요일시간대2유동인구수,남성연령대20금요일시간대3유동인구수,남성연령대20금요일시간대4유동인구수,남성연령대20금요일시간대5유동인구수,남성연령대20금요일시간대6유동인구수,남성연령대20토요일시간대1유동인구수,남성연령대20토요일시간대2유동인구수,남성연령대20토요일시간대3유동인구수,남성연령대20토요일시간대4유동인구수,남성연령대20토요일시간대5유동인구수,남성연령대20토요일시간대6유동인구수,남성연령대20일요일시간대1유동인구수,남성연령대20일요일시간대2유동인구수,남성연령대20일요일시간대3유동인구수,남성연령대20일요일시간대4유동인구수,남성연령대20일요일시간대5유동인구수,남성연령대20일요일시간대6유동인구수,남성연령대30월요일시간대1유동인구수,남성연령대30월요일시간대2유동인구수,남성연령대30월요일시간대3유동인구수,남성연령대30월요일시간대4유동인구수,남성연령대30월요일시간대5유동인구수,남성연령대30월요일시간대6유동인구수,남성연령대30화요일시간대1유동인구수,남성연령대30화요일시간대2유동인구수,남성연령대30화요일시간대3유동인구수,남성연령대30화요일시간대4유동인구수,남성연령대30화요일시간대5유동인구수,남성연령대30화요일시간대6유동인구수,남성연령대30수요일시간대1유동인구수,남성연령대30수요일시간대2유동인구수,남성연령대30수요일시간대3유동인구수,남성연령대30수요일시간대4유동인구수,남성연령대30수요일시간대5유동인구수,남성연령대30수요일시간대6유동인구수,남성연령대30목요일시간대1유동인구수,남성연령대30목요일시간대2유동인구수,남성연령대30목요일시간대3유동인구수,남성연령대30목요일시간대4유동인구수,남성연령대30목요일시간대5유동인구수,남성연령대30목요일시간대6유동인구수,남성연령대30금요일시간대1유동인구수,남성연령대30금요일시간대2유동인구수,남성연령대30금요일시간대3유동인구수,남성연령대30금요일시간대4유동인구수,남성연령대30금요일시간대5유동인구수,남성연령대30금요일시간대6유동인구수,남성연령대30토요일시간대1유동인구수,남성연령대30토요일시간대2유동인구수,남성연령대30토요일시간대3유동인구수,남성연령대30토요일시간대4유동인구수,남성연령대30토요일시간대5유동인구수,남성연령대30토요일시간대6유동인구수,남성연령대30일요일시간대1유동인구수,남성연령대30일요일시간대2유동인구수,남성연령대30일요일시간대3유동인구수,남성연령대30일요일시간대4유동인구수,남성연령대30일요일시간대5유동인구수,남성연령대30일요일시간대6유동인구수,남성연령대40월요일시간대1유동인구수,남성연령대40월요일시간대2유동인구수,남성연령대40월요일시간대3유동인구수,남성연령대40월요일시간대4유동인구수,남성연령대40월요일시간대5유동인구수,남성연령대40월요일

In [103]:
ML_model.to_pickle('merge_콜론대쉬제거_210121.pki')

In [22]:
# 콜론명 '점포수' 2개 중 한개 지워야함
ML_model = pd.read_pickle('merge_콜론대쉬제거_210121.pki')

In [23]:
for i in range(len(ML_model.columns)):
    if ML_model.columns[i] == '점포수':
        print(i,ML_model.columns[i])

31 점포수
101 점포수


In [24]:
ML_model.head(1)

,adres,도로명,상권코드명,서비스업종코드명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,상권구분코드,상권구분코드명,상권코드,서비스업종코드,분기매출금액,당월매출건수,엑스좌표값,와이좌표값,행정동코드명,임대시세1층,임대시세1층외,소득분위,점포당분기매출,임대시세전체,영업일수,가구수,소득분위점수,점포수,주중매출비율,주말매출비율,월요일매출비율,화요일매출비율,수요일매출비율,목요일매출비율,금요일매출비율,토요일매출비율,일요일매출비율,시간대0006매출비율,시간대0611매출비율,시간대1114매출비율,시간대1417매출비율,시간대1721매출비율,시간대2124매출비율,남성매출비율,여성매출비율,연령대10매출비율,연령대20매출비율,연령대30매출비율,연령대40매출비율,연령대50매출비율,연령대60이상매출비율,주중매출금액,주말매출금액,월요일매출금액,화요일매출금액,수요일매출금액,목요일매출금액,금요일매출금액,토요일매출금액,일요일매출금액,시간대0006매출금액,시간대0611매출금액,시간대1114매출금액,시간대1417매출금액,시간대1721매출금액,시간대2124매출금액,남성매출금액,여성매출금액,연령대10매출금액,연령대20매출금액,연령대30매출금액,연령대40매출금액,연령대50매출금액,연령대60이상매출금액,주중매출건수,주말매출건수,월요일매출건수,화요일매출건수,수요일매출건수,목요일매출건수,금요일매출건수,토요일매출건수,일요일매출건수,시간대건수06매출건수,시간대건수11매출건수,시간대건수14매출건수,시간대건수17매출건수,시간대건수21매출건수,시간대건수24매출건수,남성매출건수,여성매출건수,연령대10매출건수,연령대20매출건수,연령대30매출건수,연령대40매출건수,연령대50매출건수,연령대60이상매출건수,점포수,유사업종점포수,개업율,개업점포수,폐업률,폐업점포수,프랜차이즈점포수,총직장인구수,남성직장인구수,여성직장인구수,연령대10직장인구수,연령대20직장인구수,연령대30직장인구수,연령대40직장인구수,연령대50직장인구수,연령대60이상직장인구수,남성연령대10직장인구수,남성연령대20직장인구수,남성연령대30직장인구수,남성연령대40직장인구수,남성연령대50직장인구수,남성연령대60이상직장인구수,여성연령대10직장인구수,여성연령대20직장인구수,여성연령대30직장인구수,여성연령대40직장인구수,여성연령대50직장인구수,여성연령대60이상직장인구수,총유동인구수,남성유동인구수,여성유동인구수,연령대10유동인구수,연령대20유동인구수,연령대30유동인구수,연령대40유동인구수,연령대50유동인구수,연령대60이상유동인구수,시간대1유동인구수,시간대2유동인구수,시간대3유동인구수,시간대4유동인구수,시간대5유동인구수,시간대6유동인구수,월요일유동인구수,화요일유동인구수,수요일유동인구수,목요일유동인구수,금요일유동인구수,토요일유동인구수,일요일유동인구수,남성연령대10월요일시간대1유동인구수,남성연령대10월요일시간대2유동인구수,남성연령대10월요일시간대3유동인구수,남성연령대10월요일시간대4유동인구수,남성연령대10월요일시간대5유동인구수,남성연령대10월요일시간대6유동인구수,남성연령대10화요일시간대1유동인구수,남성연령대10화요일시간대2유동인구수,남성연령대10화요일시간대3유동인구수,남성연령대10화요일시간대4유동인구수,남성연령대10화요일시간대5유동인구수,남성연령대10화요일시간대6유동인구수,남성연령대10수요일시간대1유동인구수,남성연령대10수요일시간대2유동인구수,남성연령대10수요일시간대3유동인구수,남성연령대10수요일시간대4유동인구수,남성연령대10수요일시간대5유동인구수,남성연령대10수요일시간대6유동인구수,남성연령대10목요일시간대1유동인구수,남성연령대10목요일시간대2유동인구수,남성연령대10목요일시간대3유동인구수,남성연령대10목요일시간대4유동인구수,남성연령대10목요일시간대5유동인구수,남성연령대10목요일시간대6유동인구수,남성연령대10금요일시간대1유동인구수,남성연령대10금요일시간대2유동인구수,남성연령대10금요일시간대3유동인구수,남성연령대10금요일시간대4유동인구수,남성연령대10금요일시간대5유동인구수,남성연령대10금요일시간대6유동인구수,남성연령대10토요일시간대1유동인구수,남성연령대10토요일시간대2유동인구수,남성연령대10토요일시간대3유동인구수,남성연령대10토요일시간대4유동인구수,남성연령대10토요일시간대5유동인구수,남성연령대10토요일시간대6유동인구수,남성연령대10일요일시간대1유동인구수,남성연령대10일요일시간대2유동인구수,남성연령대10일요일시간대3유동인구수,남성연령대10일요일시간대4유동인구수,남성연령대10일요일시간대5유동인구수,남성연령대10일요일시간대6유동인구수,남성연령대20월요일시간대1유동인구수,남성연령대20월요일시간대2유동인구수,남성연령대20월요일시간대3유동인구수,남성연령대20월요일시간대4유동인구수,남성연령대20월요일시간대5유동인구수,남성연령대20월요일시간대6유동인구수,남성연령대20화요일시간대1유동인구수,남성연령대20화요일시간대2유동인구수,남성연령대20화요일시간대3유동인구수,남성연령대20화요일시간대4유동인구수,남성연령대20화요일시간대5유동인구수,남성연령대20화요일시간대6유동인구수,남성연령대20수요일시간대1유동인구수,남성연령대20수요일시간대2유동인구수,남성연령대20수요일시간대3유동인구수,남성연령대20수요일시간대4유동인구수,남성연령대20수요일시간대5유동인구수,남성연령대20수요일시간대6유동인구수,남성연령대20목요일시간대1유동인구수,남성연령대20목요일시간대2유동인구수,남성연령대20목요일시간대3유동인구수,남성연령대20목요일시간대4유동인구수,남성연령대20목요일시간대5유동인구수,남성연령대20목요일시간대6유동인구수,남성연령대20금요일시간대1유동인구수,남성연령대20금요일시간대2유동인구수,남성연령대20금요일시간대3유동인구수,남성연령대20금요일시간대4유동인구수,남성연령대20금요일시간대5유동인구수,남성연령대20금요일시간대6유동인구수,남성연령대20토요일시간대1유동인구수,남성연령대20토요일시간대2유동인구수,남성연령대20토요일시간대3유동인구수,남성연령대20토요일시간대4유동인구수,남성연령대20토요일시간대5유동인구수,남성연령대20토요일시간대6유동인구수,남성연령대20일요일시간대1유동인구수,남성연령대20일요일시간대2유동인구수,남성연령대20일요일시간대3유동인구수,남성연령대20일요일시간대4유동인구수,남성연령대20일요일시간대5유동인구수,남성연령대20일요일시간대6유동인구수,남성연령대30월요일시간대1유동인구수,남성연령대30월요일시간대2유동인구수,남성연령대30월요일시간대3유동인구수,남성연령대30월요일시간대4유동인구수,남성연령대30월요일시간대5유동인구수,남성연령대30월요일시간대6유동인구수,남성연령대30화요일시간대1유동인구수,남성연령대30화요일시간대2유동인구수,남성연령대30화요일시간대3유동인구수,남성연령대30화요일시간대4유동인구수,남성연령대30화요일시간대5유동인구수,남성연령대30화요일시간대6유동인구수,남성연령대30수요일시간대1유동인구수,남성연령대30수요일시간대2유동인구수,남성연령대30수요일시간대3유동인구수,남성연령대30수요일시간대4유동인구수,남성연령대30수요일시간대5유동인구수,남성연령대30수요일시간대6유동인구수,남성연령대30목요일시간대1유동인구수,남성연령대30목요일시간대2유동인구수,남성연령대30목요일시간대3유동인구수,남성연령대30목요일시간대4유동인구수,남성연령대30목요일시간대5유동인구수,남성연령대30목요일시간대6유동인구수,남성연령대30금요일시간대1유동인구수,남성연령대30금요일시간대2유동인구수,남성연령대30금요일시간대3유동인구수,남성연령대30금요일시간대4유동인구수,남성연령대30금요일시간대5유동인구수,남성연령대30금요일시간대6유동인구수,남성연령대30토요일시간대1유동인구수,남성연령대30토요일시간대2유동인구수,남성연령대30토요일시간대3유동인구수,남성연령대30토요일시간대4유동인구수,남성연령대30토요일시간대5유동인구수,남성연령대30토요일시간대6유동인구수,남성연령대30일요일시간대1유동인구수,남성연령대30일요일시간대2유동인구수,남성연령대30일요일시간대3유동인구수,남성연령대30일요일시간대4유동인구수,남성연령대30일요일시간대5유동인구수,남성연령대30일요일시간대6유동인구수,남성연령대40월요일시간대1유동인구수,남성연령대40월요일시간대2유동인구수,남성연령대40월요일시간대3유동인구수,남성연령대40월요일시간대4유동인구수,남성연령대40월요일시간대5유동인구수,남성연령대40월요일

In [43]:
x = ML_model.iloc[:,31]

In [46]:
ML_model.drop([ML_model.columns[101]], axis=1, inplace=True)
ML_model.head(1)

,adres,도로명,상권코드명,서비스업종코드명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,상권구분코드,상권구분코드명,상권코드,서비스업종코드,분기매출금액,당월매출건수,엑스좌표값,와이좌표값,행정동코드명,임대시세1층,임대시세1층외,소득분위,점포당분기매출,임대시세전체,영업일수,가구수,소득분위점수,주중매출비율,주말매출비율,월요일매출비율,화요일매출비율,수요일매출비율,목요일매출비율,금요일매출비율,토요일매출비율,일요일매출비율,시간대0006매출비율,시간대0611매출비율,시간대1114매출비율,시간대1417매출비율,시간대1721매출비율,시간대2124매출비율,남성매출비율,여성매출비율,연령대10매출비율,연령대20매출비율,연령대30매출비율,연령대40매출비율,연령대50매출비율,연령대60이상매출비율,주중매출금액,주말매출금액,월요일매출금액,화요일매출금액,수요일매출금액,목요일매출금액,금요일매출금액,토요일매출금액,일요일매출금액,시간대0006매출금액,시간대0611매출금액,시간대1114매출금액,시간대1417매출금액,시간대1721매출금액,시간대2124매출금액,남성매출금액,여성매출금액,연령대10매출금액,연령대20매출금액,연령대30매출금액,연령대40매출금액,연령대50매출금액,연령대60이상매출금액,주중매출건수,주말매출건수,월요일매출건수,화요일매출건수,수요일매출건수,목요일매출건수,금요일매출건수,토요일매출건수,일요일매출건수,시간대건수06매출건수,시간대건수11매출건수,시간대건수14매출건수,시간대건수17매출건수,시간대건수21매출건수,시간대건수24매출건수,남성매출건수,여성매출건수,연령대10매출건수,연령대20매출건수,연령대30매출건수,연령대40매출건수,연령대50매출건수,연령대60이상매출건수,유사업종점포수,개업율,개업점포수,폐업률,폐업점포수,프랜차이즈점포수,총직장인구수,남성직장인구수,여성직장인구수,연령대10직장인구수,연령대20직장인구수,연령대30직장인구수,연령대40직장인구수,연령대50직장인구수,연령대60이상직장인구수,남성연령대10직장인구수,남성연령대20직장인구수,남성연령대30직장인구수,남성연령대40직장인구수,남성연령대50직장인구수,남성연령대60이상직장인구수,여성연령대10직장인구수,여성연령대20직장인구수,여성연령대30직장인구수,여성연령대40직장인구수,여성연령대50직장인구수,여성연령대60이상직장인구수,총유동인구수,남성유동인구수,여성유동인구수,연령대10유동인구수,연령대20유동인구수,연령대30유동인구수,연령대40유동인구수,연령대50유동인구수,연령대60이상유동인구수,시간대1유동인구수,시간대2유동인구수,시간대3유동인구수,시간대4유동인구수,시간대5유동인구수,시간대6유동인구수,월요일유동인구수,화요일유동인구수,수요일유동인구수,목요일유동인구수,금요일유동인구수,토요일유동인구수,일요일유동인구수,남성연령대10월요일시간대1유동인구수,남성연령대10월요일시간대2유동인구수,남성연령대10월요일시간대3유동인구수,남성연령대10월요일시간대4유동인구수,남성연령대10월요일시간대5유동인구수,남성연령대10월요일시간대6유동인구수,남성연령대10화요일시간대1유동인구수,남성연령대10화요일시간대2유동인구수,남성연령대10화요일시간대3유동인구수,남성연령대10화요일시간대4유동인구수,남성연령대10화요일시간대5유동인구수,남성연령대10화요일시간대6유동인구수,남성연령대10수요일시간대1유동인구수,남성연령대10수요일시간대2유동인구수,남성연령대10수요일시간대3유동인구수,남성연령대10수요일시간대4유동인구수,남성연령대10수요일시간대5유동인구수,남성연령대10수요일시간대6유동인구수,남성연령대10목요일시간대1유동인구수,남성연령대10목요일시간대2유동인구수,남성연령대10목요일시간대3유동인구수,남성연령대10목요일시간대4유동인구수,남성연령대10목요일시간대5유동인구수,남성연령대10목요일시간대6유동인구수,남성연령대10금요일시간대1유동인구수,남성연령대10금요일시간대2유동인구수,남성연령대10금요일시간대3유동인구수,남성연령대10금요일시간대4유동인구수,남성연령대10금요일시간대5유동인구수,남성연령대10금요일시간대6유동인구수,남성연령대10토요일시간대1유동인구수,남성연령대10토요일시간대2유동인구수,남성연령대10토요일시간대3유동인구수,남성연령대10토요일시간대4유동인구수,남성연령대10토요일시간대5유동인구수,남성연령대10토요일시간대6유동인구수,남성연령대10일요일시간대1유동인구수,남성연령대10일요일시간대2유동인구수,남성연령대10일요일시간대3유동인구수,남성연령대10일요일시간대4유동인구수,남성연령대10일요일시간대5유동인구수,남성연령대10일요일시간대6유동인구수,남성연령대20월요일시간대1유동인구수,남성연령대20월요일시간대2유동인구수,남성연령대20월요일시간대3유동인구수,남성연령대20월요일시간대4유동인구수,남성연령대20월요일시간대5유동인구수,남성연령대20월요일시간대6유동인구수,남성연령대20화요일시간대1유동인구수,남성연령대20화요일시간대2유동인구수,남성연령대20화요일시간대3유동인구수,남성연령대20화요일시간대4유동인구수,남성연령대20화요일시간대5유동인구수,남성연령대20화요일시간대6유동인구수,남성연령대20수요일시간대1유동인구수,남성연령대20수요일시간대2유동인구수,남성연령대20수요일시간대3유동인구수,남성연령대20수요일시간대4유동인구수,남성연령대20수요일시간대5유동인구수,남성연령대20수요일시간대6유동인구수,남성연령대20목요일시간대1유동인구수,남성연령대20목요일시간대2유동인구수,남성연령대20목요일시간대3유동인구수,남성연령대20목요일시간대4유동인구수,남성연령대20목요일시간대5유동인구수,남성연령대20목요일시간대6유동인구수,남성연령대20금요일시간대1유동인구수,남성연령대20금요일시간대2유동인구수,남성연령대20금요일시간대3유동인구수,남성연령대20금요일시간대4유동인구수,남성연령대20금요일시간대5유동인구수,남성연령대20금요일시간대6유동인구수,남성연령대20토요일시간대1유동인구수,남성연령대20토요일시간대2유동인구수,남성연령대20토요일시간대3유동인구수,남성연령대20토요일시간대4유동인구수,남성연령대20토요일시간대5유동인구수,남성연령대20토요일시간대6유동인구수,남성연령대20일요일시간대1유동인구수,남성연령대20일요일시간대2유동인구수,남성연령대20일요일시간대3유동인구수,남성연령대20일요일시간대4유동인구수,남성연령대20일요일시간대5유동인구수,남성연령대20일요일시간대6유동인구수,남성연령대30월요일시간대1유동인구수,남성연령대30월요일시간대2유동인구수,남성연령대30월요일시간대3유동인구수,남성연령대30월요일시간대4유동인구수,남성연령대30월요일시간대5유동인구수,남성연령대30월요일시간대6유동인구수,남성연령대30화요일시간대1유동인구수,남성연령대30화요일시간대2유동인구수,남성연령대30화요일시간대3유동인구수,남성연령대30화요일시간대4유동인구수,남성연령대30화요일시간대5유동인구수,남성연령대30화요일시간대6유동인구수,남성연령대30수요일시간대1유동인구수,남성연령대30수요일시간대2유동인구수,남성연령대30수요일시간대3유동인구수,남성연령대30수요일시간대4유동인구수,남성연령대30수요일시간대5유동인구수,남성연령대30수요일시간대6유동인구수,남성연령대30목요일시간대1유동인구수,남성연령대30목요일시간대2유동인구수,남성연령대30목요일시간대3유동인구수,남성연령대30목요일시간대4유동인구수,남성연령대30목요일시간대5유동인구수,남성연령대30목요일시간대6유동인구수,남성연령대30금요일시간대1유동인구수,남성연령대30금요일시간대2유동인구수,남성연령대30금요일시간대3유동인구수,남성연령대30금요일시간대4유동인구수,남성연령대30금요일시간대5유동인구수,남성연령대30금요일시간대6유동인구수,남성연령대30토요일시간대1유동인구수,남성연령대30토요일시간대2유동인구수,남성연령대30토요일시간대3유동인구수,남성연령대30토요일시간대4유동인구수,남성연령대30토요일시간대5유동인구수,남성연령대30토요일시간대6유동인구수,남성연령대30일요일시간대1유동인구수,남성연령대30일요일시간대2유동인구수,남성연령대30일요일시간대3유동인구수,남성연령대30일요일시간대4유동인구수,남성연령대30일요일시간대5유동인구수,남성연령대30일요일시간대6유동인구수,남성연령대40월요일시간대1유동인구수,남성연령대40월요일시간대2유동인구수,남성연령대40월요일시간대3유동인구수,남성연령대40월요일시간대4유동인구수,남성연령대40월요일시간대5유동인구수,남성연령대40월요일시간대6유동인구

In [47]:
ML_model.shape

(1032749, 712)

In [48]:
ML_model.insert(31,"점포수",x,True)

In [49]:
ML_model.head(1)

,adres,도로명,상권코드명,서비스업종코드명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,상권구분코드,상권구분코드명,상권코드,서비스업종코드,분기매출금액,당월매출건수,엑스좌표값,와이좌표값,행정동코드명,임대시세1층,임대시세1층외,소득분위,점포당분기매출,임대시세전체,영업일수,가구수,소득분위점수,점포수,주중매출비율,주말매출비율,월요일매출비율,화요일매출비율,수요일매출비율,목요일매출비율,금요일매출비율,토요일매출비율,일요일매출비율,시간대0006매출비율,시간대0611매출비율,시간대1114매출비율,시간대1417매출비율,시간대1721매출비율,시간대2124매출비율,남성매출비율,여성매출비율,연령대10매출비율,연령대20매출비율,연령대30매출비율,연령대40매출비율,연령대50매출비율,연령대60이상매출비율,주중매출금액,주말매출금액,월요일매출금액,화요일매출금액,수요일매출금액,목요일매출금액,금요일매출금액,토요일매출금액,일요일매출금액,시간대0006매출금액,시간대0611매출금액,시간대1114매출금액,시간대1417매출금액,시간대1721매출금액,시간대2124매출금액,남성매출금액,여성매출금액,연령대10매출금액,연령대20매출금액,연령대30매출금액,연령대40매출금액,연령대50매출금액,연령대60이상매출금액,주중매출건수,주말매출건수,월요일매출건수,화요일매출건수,수요일매출건수,목요일매출건수,금요일매출건수,토요일매출건수,일요일매출건수,시간대건수06매출건수,시간대건수11매출건수,시간대건수14매출건수,시간대건수17매출건수,시간대건수21매출건수,시간대건수24매출건수,남성매출건수,여성매출건수,연령대10매출건수,연령대20매출건수,연령대30매출건수,연령대40매출건수,연령대50매출건수,연령대60이상매출건수,유사업종점포수,개업율,개업점포수,폐업률,폐업점포수,프랜차이즈점포수,총직장인구수,남성직장인구수,여성직장인구수,연령대10직장인구수,연령대20직장인구수,연령대30직장인구수,연령대40직장인구수,연령대50직장인구수,연령대60이상직장인구수,남성연령대10직장인구수,남성연령대20직장인구수,남성연령대30직장인구수,남성연령대40직장인구수,남성연령대50직장인구수,남성연령대60이상직장인구수,여성연령대10직장인구수,여성연령대20직장인구수,여성연령대30직장인구수,여성연령대40직장인구수,여성연령대50직장인구수,여성연령대60이상직장인구수,총유동인구수,남성유동인구수,여성유동인구수,연령대10유동인구수,연령대20유동인구수,연령대30유동인구수,연령대40유동인구수,연령대50유동인구수,연령대60이상유동인구수,시간대1유동인구수,시간대2유동인구수,시간대3유동인구수,시간대4유동인구수,시간대5유동인구수,시간대6유동인구수,월요일유동인구수,화요일유동인구수,수요일유동인구수,목요일유동인구수,금요일유동인구수,토요일유동인구수,일요일유동인구수,남성연령대10월요일시간대1유동인구수,남성연령대10월요일시간대2유동인구수,남성연령대10월요일시간대3유동인구수,남성연령대10월요일시간대4유동인구수,남성연령대10월요일시간대5유동인구수,남성연령대10월요일시간대6유동인구수,남성연령대10화요일시간대1유동인구수,남성연령대10화요일시간대2유동인구수,남성연령대10화요일시간대3유동인구수,남성연령대10화요일시간대4유동인구수,남성연령대10화요일시간대5유동인구수,남성연령대10화요일시간대6유동인구수,남성연령대10수요일시간대1유동인구수,남성연령대10수요일시간대2유동인구수,남성연령대10수요일시간대3유동인구수,남성연령대10수요일시간대4유동인구수,남성연령대10수요일시간대5유동인구수,남성연령대10수요일시간대6유동인구수,남성연령대10목요일시간대1유동인구수,남성연령대10목요일시간대2유동인구수,남성연령대10목요일시간대3유동인구수,남성연령대10목요일시간대4유동인구수,남성연령대10목요일시간대5유동인구수,남성연령대10목요일시간대6유동인구수,남성연령대10금요일시간대1유동인구수,남성연령대10금요일시간대2유동인구수,남성연령대10금요일시간대3유동인구수,남성연령대10금요일시간대4유동인구수,남성연령대10금요일시간대5유동인구수,남성연령대10금요일시간대6유동인구수,남성연령대10토요일시간대1유동인구수,남성연령대10토요일시간대2유동인구수,남성연령대10토요일시간대3유동인구수,남성연령대10토요일시간대4유동인구수,남성연령대10토요일시간대5유동인구수,남성연령대10토요일시간대6유동인구수,남성연령대10일요일시간대1유동인구수,남성연령대10일요일시간대2유동인구수,남성연령대10일요일시간대3유동인구수,남성연령대10일요일시간대4유동인구수,남성연령대10일요일시간대5유동인구수,남성연령대10일요일시간대6유동인구수,남성연령대20월요일시간대1유동인구수,남성연령대20월요일시간대2유동인구수,남성연령대20월요일시간대3유동인구수,남성연령대20월요일시간대4유동인구수,남성연령대20월요일시간대5유동인구수,남성연령대20월요일시간대6유동인구수,남성연령대20화요일시간대1유동인구수,남성연령대20화요일시간대2유동인구수,남성연령대20화요일시간대3유동인구수,남성연령대20화요일시간대4유동인구수,남성연령대20화요일시간대5유동인구수,남성연령대20화요일시간대6유동인구수,남성연령대20수요일시간대1유동인구수,남성연령대20수요일시간대2유동인구수,남성연령대20수요일시간대3유동인구수,남성연령대20수요일시간대4유동인구수,남성연령대20수요일시간대5유동인구수,남성연령대20수요일시간대6유동인구수,남성연령대20목요일시간대1유동인구수,남성연령대20목요일시간대2유동인구수,남성연령대20목요일시간대3유동인구수,남성연령대20목요일시간대4유동인구수,남성연령대20목요일시간대5유동인구수,남성연령대20목요일시간대6유동인구수,남성연령대20금요일시간대1유동인구수,남성연령대20금요일시간대2유동인구수,남성연령대20금요일시간대3유동인구수,남성연령대20금요일시간대4유동인구수,남성연령대20금요일시간대5유동인구수,남성연령대20금요일시간대6유동인구수,남성연령대20토요일시간대1유동인구수,남성연령대20토요일시간대2유동인구수,남성연령대20토요일시간대3유동인구수,남성연령대20토요일시간대4유동인구수,남성연령대20토요일시간대5유동인구수,남성연령대20토요일시간대6유동인구수,남성연령대20일요일시간대1유동인구수,남성연령대20일요일시간대2유동인구수,남성연령대20일요일시간대3유동인구수,남성연령대20일요일시간대4유동인구수,남성연령대20일요일시간대5유동인구수,남성연령대20일요일시간대6유동인구수,남성연령대30월요일시간대1유동인구수,남성연령대30월요일시간대2유동인구수,남성연령대30월요일시간대3유동인구수,남성연령대30월요일시간대4유동인구수,남성연령대30월요일시간대5유동인구수,남성연령대30월요일시간대6유동인구수,남성연령대30화요일시간대1유동인구수,남성연령대30화요일시간대2유동인구수,남성연령대30화요일시간대3유동인구수,남성연령대30화요일시간대4유동인구수,남성연령대30화요일시간대5유동인구수,남성연령대30화요일시간대6유동인구수,남성연령대30수요일시간대1유동인구수,남성연령대30수요일시간대2유동인구수,남성연령대30수요일시간대3유동인구수,남성연령대30수요일시간대4유동인구수,남성연령대30수요일시간대5유동인구수,남성연령대30수요일시간대6유동인구수,남성연령대30목요일시간대1유동인구수,남성연령대30목요일시간대2유동인구수,남성연령대30목요일시간대3유동인구수,남성연령대30목요일시간대4유동인구수,남성연령대30목요일시간대5유동인구수,남성연령대30목요일시간대6유동인구수,남성연령대30금요일시간대1유동인구수,남성연령대30금요일시간대2유동인구수,남성연령대30금요일시간대3유동인구수,남성연령대30금요일시간대4유동인구수,남성연령대30금요일시간대5유동인구수,남성연령대30금요일시간대6유동인구수,남성연령대30토요일시간대1유동인구수,남성연령대30토요일시간대2유동인구수,남성연령대30토요일시간대3유동인구수,남성연령대30토요일시간대4유동인구수,남성연령대30토요일시간대5유동인구수,남성연령대30토요일시간대6유동인구수,남성연령대30일요일시간대1유동인구수,남성연령대30일요일시간대2유동인구수,남성연령대30일요일시간대3유동인구수,남성연령대30일요일시간대4유동인구수,남성연령대30일요일시간대5유동인구수,남성연령대30일요일시간대6유동인구수,남성연령대40월요일시간대1유동인구수,남성연령대40월요일시간대2유동인구수,남성연령대40월요일시간대3유동인구수,남성연령대40월요일시간대4유동인구수,남성연령대40월요일시간대5유동인구수,남성연령대40월요일시간대6

In [51]:
ML_model.점포수

0           5.0
1           5.0
2          15.0
3          15.0
4          15.0
           ... 
1032744     9.0
1032745     9.0
1032746     9.0
1032747     9.0
1032748     9.0
Name: 점포수, Length: 1032749, dtype: float64

In [52]:
ML_model.to_pickle('merge_콜론대쉬제거_210121.pki')

In [2]:
ML_model= pd.read_pickle('merge_콜론대쉬제거_210121.pki')
ML_model.영업일계산기준일

0         2020-12-01
1         2020-12-01
2         2016-04-01
3         2020-12-01
4         2009-09-01
             ...    
1032744   2016-12-01
1032745   2014-12-01
1032746   2014-09-01
1032747   2011-06-01
1032748   2008-04-01
Name: 영업일계산기준일, Length: 1032749, dtype: datetime64[ns]

In [6]:
ML_model.영업일계산기준일=20201231

In [7]:
ML_model.영업일계산기준일.unique()

array([20201231])

In [8]:
ML_model

,adres,도로명,상권코드명,서비스업종코드명,소분류업종,storNm,totalCount,coordx,coordy,x,y,opbizYm,clsbizYm,영업일계산기준일,상권구분코드,상권구분코드명,상권코드,서비스업종코드,분기매출금액,당월매출건수,엑스좌표값,와이좌표값,행정동코드명,임대시세1층,임대시세1층외,소득분위,점포당분기매출,임대시세전체,영업일수,가구수,소득분위점수,점포수,주중매출비율,주말매출비율,월요일매출비율,화요일매출비율,수요일매출비율,목요일매출비율,금요일매출비율,토요일매출비율,일요일매출비율,시간대0006매출비율,시간대0611매출비율,시간대1114매출비율,시간대1417매출비율,시간대1721매출비율,시간대2124매출비율,남성매출비율,여성매출비율,연령대10매출비율,연령대20매출비율,연령대30매출비율,연령대40매출비율,연령대50매출비율,연령대60이상매출비율,주중매출금액,주말매출금액,월요일매출금액,화요일매출금액,수요일매출금액,목요일매출금액,금요일매출금액,토요일매출금액,일요일매출금액,시간대0006매출금액,시간대0611매출금액,시간대1114매출금액,시간대1417매출금액,시간대1721매출금액,시간대2124매출금액,남성매출금액,여성매출금액,연령대10매출금액,연령대20매출금액,연령대30매출금액,연령대40매출금액,연령대50매출금액,연령대60이상매출금액,주중매출건수,주말매출건수,월요일매출건수,화요일매출건수,수요일매출건수,목요일매출건수,금요일매출건수,토요일매출건수,일요일매출건수,시간대건수06매출건수,시간대건수11매출건수,시간대건수14매출건수,시간대건수17매출건수,시간대건수21매출건수,시간대건수24매출건수,남성매출건수,여성매출건수,연령대10매출건수,연령대20매출건수,연령대30매출건수,연령대40매출건수,연령대50매출건수,연령대60이상매출건수,유사업종점포수,개업율,개업점포수,폐업률,폐업점포수,프랜차이즈점포수,총직장인구수,남성직장인구수,여성직장인구수,연령대10직장인구수,연령대20직장인구수,연령대30직장인구수,연령대40직장인구수,연령대50직장인구수,연령대60이상직장인구수,남성연령대10직장인구수,남성연령대20직장인구수,남성연령대30직장인구수,남성연령대40직장인구수,남성연령대50직장인구수,남성연령대60이상직장인구수,여성연령대10직장인구수,여성연령대20직장인구수,여성연령대30직장인구수,여성연령대40직장인구수,여성연령대50직장인구수,여성연령대60이상직장인구수,총유동인구수,남성유동인구수,여성유동인구수,연령대10유동인구수,연령대20유동인구수,연령대30유동인구수,연령대40유동인구수,연령대50유동인구수,연령대60이상유동인구수,시간대1유동인구수,시간대2유동인구수,시간대3유동인구수,시간대4유동인구수,시간대5유동인구수,시간대6유동인구수,월요일유동인구수,화요일유동인구수,수요일유동인구수,목요일유동인구수,금요일유동인구수,토요일유동인구수,일요일유동인구수,남성연령대10월요일시간대1유동인구수,남성연령대10월요일시간대2유동인구수,남성연령대10월요일시간대3유동인구수,남성연령대10월요일시간대4유동인구수,남성연령대10월요일시간대5유동인구수,남성연령대10월요일시간대6유동인구수,남성연령대10화요일시간대1유동인구수,남성연령대10화요일시간대2유동인구수,남성연령대10화요일시간대3유동인구수,남성연령대10화요일시간대4유동인구수,남성연령대10화요일시간대5유동인구수,남성연령대10화요일시간대6유동인구수,남성연령대10수요일시간대1유동인구수,남성연령대10수요일시간대2유동인구수,남성연령대10수요일시간대3유동인구수,남성연령대10수요일시간대4유동인구수,남성연령대10수요일시간대5유동인구수,남성연령대10수요일시간대6유동인구수,남성연령대10목요일시간대1유동인구수,남성연령대10목요일시간대2유동인구수,남성연령대10목요일시간대3유동인구수,남성연령대10목요일시간대4유동인구수,남성연령대10목요일시간대5유동인구수,남성연령대10목요일시간대6유동인구수,남성연령대10금요일시간대1유동인구수,남성연령대10금요일시간대2유동인구수,남성연령대10금요일시간대3유동인구수,남성연령대10금요일시간대4유동인구수,남성연령대10금요일시간대5유동인구수,남성연령대10금요일시간대6유동인구수,남성연령대10토요일시간대1유동인구수,남성연령대10토요일시간대2유동인구수,남성연령대10토요일시간대3유동인구수,남성연령대10토요일시간대4유동인구수,남성연령대10토요일시간대5유동인구수,남성연령대10토요일시간대6유동인구수,남성연령대10일요일시간대1유동인구수,남성연령대10일요일시간대2유동인구수,남성연령대10일요일시간대3유동인구수,남성연령대10일요일시간대4유동인구수,남성연령대10일요일시간대5유동인구수,남성연령대10일요일시간대6유동인구수,남성연령대20월요일시간대1유동인구수,남성연령대20월요일시간대2유동인구수,남성연령대20월요일시간대3유동인구수,남성연령대20월요일시간대4유동인구수,남성연령대20월요일시간대5유동인구수,남성연령대20월요일시간대6유동인구수,남성연령대20화요일시간대1유동인구수,남성연령대20화요일시간대2유동인구수,남성연령대20화요일시간대3유동인구수,남성연령대20화요일시간대4유동인구수,남성연령대20화요일시간대5유동인구수,남성연령대20화요일시간대6유동인구수,남성연령대20수요일시간대1유동인구수,남성연령대20수요일시간대2유동인구수,남성연령대20수요일시간대3유동인구수,남성연령대20수요일시간대4유동인구수,남성연령대20수요일시간대5유동인구수,남성연령대20수요일시간대6유동인구수,남성연령대20목요일시간대1유동인구수,남성연령대20목요일시간대2유동인구수,남성연령대20목요일시간대3유동인구수,남성연령대20목요일시간대4유동인구수,남성연령대20목요일시간대5유동인구수,남성연령대20목요일시간대6유동인구수,남성연령대20금요일시간대1유동인구수,남성연령대20금요일시간대2유동인구수,남성연령대20금요일시간대3유동인구수,남성연령대20금요일시간대4유동인구수,남성연령대20금요일시간대5유동인구수,남성연령대20금요일시간대6유동인구수,남성연령대20토요일시간대1유동인구수,남성연령대20토요일시간대2유동인구수,남성연령대20토요일시간대3유동인구수,남성연령대20토요일시간대4유동인구수,남성연령대20토요일시간대5유동인구수,남성연령대20토요일시간대6유동인구수,남성연령대20일요일시간대1유동인구수,남성연령대20일요일시간대2유동인구수,남성연령대20일요일시간대3유동인구수,남성연령대20일요일시간대4유동인구수,남성연령대20일요일시간대5유동인구수,남성연령대20일요일시간대6유동인구수,남성연령대30월요일시간대1유동인구수,남성연령대30월요일시간대2유동인구수,남성연령대30월요일시간대3유동인구수,남성연령대30월요일시간대4유동인구수,남성연령대30월요일시간대5유동인구수,남성연령대30월요일시간대6유동인구수,남성연령대30화요일시간대1유동인구수,남성연령대30화요일시간대2유동인구수,남성연령대30화요일시간대3유동인구수,남성연령대30화요일시간대4유동인구수,남성연령대30화요일시간대5유동인구수,남성연령대30화요일시간대6유동인구수,남성연령대30수요일시간대1유동인구수,남성연령대30수요일시간대2유동인구수,남성연령대30수요일시간대3유동인구수,남성연령대30수요일시간대4유동인구수,남성연령대30수요일시간대5유동인구수,남성연령대30수요일시간대6유동인구수,남성연령대30목요일시간대1유동인구수,남성연령대30목요일시간대2유동인구수,남성연령대30목요일시간대3유동인구수,남성연령대30목요일시간대4유동인구수,남성연령대30목요일시간대5유동인구수,남성연령대30목요일시간대6유동인구수,남성연령대30금요일시간대1유동인구수,남성연령대30금요일시간대2유동인구수,남성연령대30금요일시간대3유동인구수,남성연령대30금요일시간대4유동인구수,남성연령대30금요일시간대5유동인구수,남성연령대30금요일시간대6유동인구수,남성연령대30토요일시간대1유동인구수,남성연령대30토요일시간대2유동인구수,남성연령대30토요일시간대3유동인구수,남성연령대30토요일시간대4유동인구수,남성연령대30토요일시간대5유동인구수,남성연령대30토요일시간대6유동인구수,남성연령대30일요일시간대1유동인구수,남성연령대30일요일시간대2유동인구수,남성연령대30일요일시간대3유동인구수,남성연령대30일요일시간대4유동인구수,남성연령대30일요일시간대5유동인구수,남성연령대30일요일시간대6유동인구수,남성연령대40월요일시간대1유동인구수,남성연령대40월요일시간대2유동인구수,남성연령대40월요일시간대3유동인구수,남성연령대40월요일시간대4유동인구수,남성연령대40월요일시간대5유동인구수,남성연령대40월요일시간대6